In [1]:
# Dependencies
import requests
from requests.exceptions import HTTPError
import json
import pandas as pd
import numpy as np
import re
import io
from config import api_key_prism
from collections import OrderedDict
from pandas.io.json import json_normalize  
import time 
import urllib.request

### Community Request in Prism
* Starting from JSON file
* Not yet complete until records are available in Prism Communities

In [106]:
## Open the DigitalHub Collections JSON file

import json

with open('data\digitalhub collections.json') as f:
    digitalhub_json_data = json.load(f)   

    print(digitalhub_json_data)

[{'community': {'access': {'member_policy': 'open', 'owned_by': {'akl5916': 'ashley.knudson@northwestern.edu', 'ccb638': 'bcc@northwestern.edu', 'tlm488': 't-brannon@northwestern.edu'}, 'record_policy': 'open', 'visibility': 'public'}, 'slug': 'biostatistics-collaboration-center-lecture-series', 'metadata': {'description': 'Free introductory lectures on biostatistics in medical research by the Biostatistics Collaboration Center.', 'title': 'Biostatistics Collaboration Center Lecture Series', 'type': 'project', 'website': 'http://www.feinberg.northwestern.edu/sites/bcc/education/lecture/2016.html'}}, 'dh_id': '2cc92425-b656-47ea-a3b4-825405ee6088'}, {'community': {'access': {'member_policy': 'open', 'owned_by': {'kah2923': 'keith.herzog@northwestern.edu', 'gmr244': 'grisel.robles-schrader@northwestern.edu'}, 'record_policy': 'open', 'visibility': 'public'}, 'slug': 'center-for-community-health', 'metadata': {'title': 'Center for Community Health', 'type': 'organization', 'website': 'htt

In [107]:
## Convert the DigitalHub Communities json file to a dataframe

digitalhub_json_collection_df = pd.DataFrame.from_dict(json_normalize(digitalhub_json_data, max_level=2))
digitalhub_json_collection_df.head(10)

C:\Users\keg827\AppData\Local\Temp\ipykernel_30316\4069261026.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  digitalhub_json_collection_df = pd.DataFrame.from_dict(json_normalize(digitalhub_json_data, max_level=2))


,dh_id,community.access.member_policy,community.access.owned_by,community.access.record_policy,community.access.visibility,community.slug,community.metadata.description,community.metadata.title,community.metadata.type,community.metadata.website,community.access.curator
0,2cc92425-b656-47ea-a3b4-825405ee6088,open,"{'akl5916': 'ashley.knudson@northwestern.edu',...",open,public,biostatistics-collaboration-center-lecture-series,Free introductory lectures on biostatistics in...,Biostatistics Collaboration Center Lecture Series,project,http://www.feinberg.northwestern.edu/sites/bcc...,NaN
1,ae0b945c-d0d4-45bb-a0fc-263c7afca49e,open,"{'kah2923': 'keith.herzog@northwestern.edu', '...",open,public,center-for-community-health,NaN,Center for Community Health,organization,https://www.feinberg.northwestern.edu/sites/cch/,NaN
2,2828d181-5de7-4567-ba8d-d6bdb72b625f,open,"{'jbs642': 'justin.starren@northwestern.edu', ...",open,public,center-for-biomedical-informatics-and-data-sci...,The Center for Data Science and Informatics (C...,Center for Biomedical Informatics and Data Sci...,organization,https://www.feinberg.northwestern.edu/sites/au...,NaN
3,44857499-5126-47bd-a226-5fbf291abaf6,open,"{'bkm413': 'brigid.smith@northwestern.edu', 'l...",open,public,dialogues-in-oncofertility,Dialogues in Oncofertility is a record of scho...,Dialogues in Oncofertility,project,https://oncofertility.msu.edu/,NaN
4,fj2362114,open,{'kal660': 'katherine.lattal@northwestern.edu'...,open,public,galter-health-sciences-library-and-learning-ce...,The Galter Health Sciences Library & Learning ...,Galter Health Sciences Library and Learning Ce...,organization,https://galter.northwestern.edu,NaN
5,40acd700-b850-4e7b-a650-0535de84ab6b,open,"{'keg827': 'karen.gutzman@northwestern.edu', '...",open,public,masters-in-public-health-ce-papers,Culminating Experience (CE) papers submitted i...,Masters in Public Health CE Papers,project,https://www.feinberg.northwestern.edu/sites/mph/,NaN
6,,open,"{'keg827': 'karen.gutzman@northwestern.edu', '...",open,public,researchers-collections,NaN,Researchers' Collections,topic,NaN,"{'bbo067': 'bbk@northwestern.edu', 'cbd468': '..."
7,b0375b45-0b95-4bf8-9ee7-7df4d6fb47e4,open,"{'klh183': 'kristi.holmes@northwestern.edu', '...",open,public,national-center-for-data-to-health,The National Center for Data to Health (CD2H) ...,The National Center for Data to Health,organization,https://cd2h.org/,NaN
8,hq37vn62g,open,{'nds616': 'nicholas.soulakis@northwestern.edu'},open,public,soulakis-lab,"Articles, news, and documents not indexed in P...",Soulakis Lab,organization,NaN,NaN
9,2e61510b-939f-4637-9fd9-c31f1013c661,open,"{'dcm863': 'd-mohr@northwestern.edu', 'ems932'...",open,public,center-for-behavioral-intervention-technologies,NaN,Center for Behavioral Intervention Technologies,organization,http://cbits.northwestern.edu,NaN


In [108]:
## Add a column in the DigitalHub Collections JSON to include a url to the new Prism slug for the community

url= "https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/"
digitalhub_json_collection_df["prism_test_url"] = url + digitalhub_json_collection_df['community.slug'].astype(str)

In [109]:
## Export the DigitalHub Collection JSON file to excel, without the Pandas index, but with the header

digitalhub_json_collection_df.to_excel("outputs/digitalhub_json_collection_df.xlsx", index=False, header=True)

In [110]:
## Each collection is a dictionary, so loop through to find slugs and add to a list.
slugs = []

for item in digitalhub_json_data:
    slug = item['community']['slug']
    slugs.append(slug)

print(len(slugs))
print(slugs)

## Each collection is a dictionary, so loop through to find dh_ids and add to a list.

dh_ids = []

for item in digitalhub_json_data:
    dh_id = item['dh_id']
    dh_ids.append(dh_id)

print(len(dh_ids))
print(dh_ids)

36
['biostatistics-collaboration-center-lecture-series', 'center-for-community-health', 'center-for-biomedical-informatics-and-data-science', 'dialogues-in-oncofertility', 'galter-health-sciences-library-and-learning-center', 'masters-in-public-health-ce-papers', 'researchers-collections', 'national-center-for-data-to-health', 'soulakis-lab', 'center-for-behavioral-intervention-technologies', 'center-for-prevention-implementation-methodology', 'cts-personas', 'northwestern-elements', 'pro-implementation-planning-toolkit', 'history-of-medicine-projects-presentations-and-exhibits', 'prevention-science-methodology-group', 'pediatric-neurology-briefs', 'science-in-society-scientific-images-contest', 'g-v-black-papers', 'nucats-grants-repository', 'communication-bridge', 'endowed-professorship-biographies', '12th-general-hospital-collection', 'history-of-feinberg-school-of-medicine', 'galter-library-audio-video-archives', 'northwestern-pepper-center', 'nucats-series-on-developing-and-enhanc

In [111]:
## Simple API request to Prism for one known slug

slug = 'biostatistics-collaboration-center-lecture-series'

url = f'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/{slug}'

params={ 
        'access_token': api_key_prism
       }

## 'com_id': "2cc92425-b656-47ea-a3b4-825405ee6088"

## Make the API request 
single_slug_response = requests.get(url, params=params)
print("This is a single slug url", single_slug_response.url)
print(single_slug_response.status_code)

## Convert single_slug_response to dict using .json()
single_slug_dict =  single_slug_response.json()
print("This is a single slug dict", single_slug_dict)



This is a single slug url https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/biostatistics-collaboration-center-lecture-series?access_token=mgS5In0TwRjbTbfr1Z72Pbj8bgAFVE4kMrOMNkw0LOLJgwzJK9sHcoOwColI
200
This is a single slug dict {'links': {'self': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e0cbc1d1-eea8-49bb-bab6-fd3227adffe1', 'self_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/biostatistics-collaboration-center-lecture-series', 'settings_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/biostatistics-collaboration-center-lecture-series/settings', 'logo': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e0cbc1d1-eea8-49bb-bab6-fd3227adffe1/logo', 'rename': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e0cbc1d1-eea8-49bb-bab6-fd3227adffe1/rename', 'members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e0cbc1d1-eea8-49bb-bab6-fd3227adffe1/members', 'public_members': 'https://

In [112]:
## Simple API request to Prism for all known slugs

multiple_slug_list = []


def get_request(slugs, multiple_slug_list):

 
    ## Create URL for request
    for slug in slugs:
        print(slug)

      
        url = f'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/{slug}'

        params={
                'all_versions': 'false', 
                'access_token': api_key_prism,
                }

        ## Make the API request 
        single_slug_response = requests.get(url, params=params)
        print("This is a single slug url", single_slug_response.url)
        print(single_slug_response.status_code)

        ## Convert single_slug_response to dict using .json()
        single_slug_dict =  single_slug_response.json()
        print("This is a single slug dict", single_slug_dict)
        
        #Append each single_slug_response response to multiple_slug_list to create a list of dictionaries
        #print(type(single_slug_dict)) 
        #print(single_slug_dict)
        multiple_slug_list.append(single_slug_dict.copy())


    return multiple_slug_list


get_request(slugs, multiple_slug_list)


## Resources
## https://inveniordm.docs.cern.ch/reference/rest_api_communities/
## https://realpython.com/python-api/#request-and-response
## http://www.compciv.org/guides/python/how-tos/creating-proper-url-query-strings/
## https://stackoverflow.com/questions/17788445/constructing-requests-with-url-query-string-in-python
## Using paginated APIs (4 ways): https://www.youtube.com/watch?v=4Fdyft-ky0w

biostatistics-collaboration-center-lecture-series
This is a single slug url https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/biostatistics-collaboration-center-lecture-series?all_versions=false&access_token=mgS5In0TwRjbTbfr1Z72Pbj8bgAFVE4kMrOMNkw0LOLJgwzJK9sHcoOwColI
200
This is a single slug dict {'links': {'self': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e0cbc1d1-eea8-49bb-bab6-fd3227adffe1', 'self_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/biostatistics-collaboration-center-lecture-series', 'settings_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/biostatistics-collaboration-center-lecture-series/settings', 'logo': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e0cbc1d1-eea8-49bb-bab6-fd3227adffe1/logo', 'rename': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e0cbc1d1-eea8-49bb-bab6-fd3227adffe1/rename', 'members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e0cbc

This is a single slug url https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/galter-health-sciences-library-and-learning-center?all_versions=false&access_token=mgS5In0TwRjbTbfr1Z72Pbj8bgAFVE4kMrOMNkw0LOLJgwzJK9sHcoOwColI
200
This is a single slug dict {'links': {'self': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/df884fce-a826-4e57-9d6f-33b092757fa3', 'self_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/galter-health-sciences-library-and-learning-center', 'settings_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/galter-health-sciences-library-and-learning-center/settings', 'logo': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/df884fce-a826-4e57-9d6f-33b092757fa3/logo', 'rename': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/df884fce-a826-4e57-9d6f-33b092757fa3/rename', 'members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/df884fce-a826-4e57-9d6f-33b092757fa3/members', 'publ

This is a single slug url https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/soulakis-lab?all_versions=false&access_token=mgS5In0TwRjbTbfr1Z72Pbj8bgAFVE4kMrOMNkw0LOLJgwzJK9sHcoOwColI
200
This is a single slug dict {'links': {'self': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/69ebb948-0b12-4370-b7d3-96992dc2f104', 'self_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/soulakis-lab', 'settings_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/soulakis-lab/settings', 'logo': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/69ebb948-0b12-4370-b7d3-96992dc2f104/logo', 'rename': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/69ebb948-0b12-4370-b7d3-96992dc2f104/rename', 'members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/69ebb948-0b12-4370-b7d3-96992dc2f104/members', 'public_members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/69ebb948-0b12-4370-b7d3-96992dc2f104/me

This is a single slug url https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/pro-implementation-planning-toolkit?all_versions=false&access_token=mgS5In0TwRjbTbfr1Z72Pbj8bgAFVE4kMrOMNkw0LOLJgwzJK9sHcoOwColI
200
This is a single slug dict {'links': {'self': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/0a8a1f45-c27d-4f73-8e0d-b527b5250353', 'self_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/pro-implementation-planning-toolkit', 'settings_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/pro-implementation-planning-toolkit/settings', 'logo': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/0a8a1f45-c27d-4f73-8e0d-b527b5250353/logo', 'rename': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/0a8a1f45-c27d-4f73-8e0d-b527b5250353/rename', 'members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/0a8a1f45-c27d-4f73-8e0d-b527b5250353/members', 'public_members': 'https://vtfsmghslrepo02.fsm.nor

This is a single slug url https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/science-in-society-scientific-images-contest?all_versions=false&access_token=mgS5In0TwRjbTbfr1Z72Pbj8bgAFVE4kMrOMNkw0LOLJgwzJK9sHcoOwColI
200
This is a single slug dict {'links': {'self': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/95795f00-0c57-47d1-8210-2e2e664e5a5d', 'self_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/science-in-society-scientific-images-contest', 'settings_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/science-in-society-scientific-images-contest/settings', 'logo': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/95795f00-0c57-47d1-8210-2e2e664e5a5d/logo', 'rename': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/95795f00-0c57-47d1-8210-2e2e664e5a5d/rename', 'members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/95795f00-0c57-47d1-8210-2e2e664e5a5d/members', 'public_members': 'http

This is a single slug url https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/endowed-professorship-biographies?all_versions=false&access_token=mgS5In0TwRjbTbfr1Z72Pbj8bgAFVE4kMrOMNkw0LOLJgwzJK9sHcoOwColI
200
This is a single slug dict {'links': {'self': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/65da9359-ce89-4717-be9e-9743e4cac174', 'self_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/endowed-professorship-biographies', 'settings_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/endowed-professorship-biographies/settings', 'logo': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/65da9359-ce89-4717-be9e-9743e4cac174/logo', 'rename': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/65da9359-ce89-4717-be9e-9743e4cac174/rename', 'members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/65da9359-ce89-4717-be9e-9743e4cac174/members', 'public_members': 'https://vtfsmghslrepo02.fsm.northwest

This is a single slug url https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/northwestern-pepper-center?all_versions=false&access_token=mgS5In0TwRjbTbfr1Z72Pbj8bgAFVE4kMrOMNkw0LOLJgwzJK9sHcoOwColI
200
This is a single slug dict {'links': {'self': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/49af5b57-1788-4727-9b76-75899844b852', 'self_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/northwestern-pepper-center', 'settings_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/northwestern-pepper-center/settings', 'logo': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/49af5b57-1788-4727-9b76-75899844b852/logo', 'rename': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/49af5b57-1788-4727-9b76-75899844b852/rename', 'members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/49af5b57-1788-4727-9b76-75899844b852/members', 'public_members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communiti

This is a single slug url https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/charting-a-new-course-practical-data-visualization?all_versions=false&access_token=mgS5In0TwRjbTbfr1Z72Pbj8bgAFVE4kMrOMNkw0LOLJgwzJK9sHcoOwColI
200
This is a single slug dict {'links': {'self': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e727ce14-b03b-4ab9-a6a8-a6c02c25e15e', 'self_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/charting-a-new-course-practical-data-visualization', 'settings_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/charting-a-new-course-practical-data-visualization/settings', 'logo': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e727ce14-b03b-4ab9-a6a8-a6c02c25e15e/logo', 'rename': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e727ce14-b03b-4ab9-a6a8-a6c02c25e15e/rename', 'members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e727ce14-b03b-4ab9-a6a8-a6c02c25e15e/members', 'publ

This is a single slug url https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/food-allergy-data-dictionary?all_versions=false&access_token=mgS5In0TwRjbTbfr1Z72Pbj8bgAFVE4kMrOMNkw0LOLJgwzJK9sHcoOwColI
200
This is a single slug dict {'links': {'self': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/f881b5bc-5f27-434a-8ad9-a2d5f06964a9', 'self_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/food-allergy-data-dictionary', 'settings_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/food-allergy-data-dictionary/settings', 'logo': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/f881b5bc-5f27-434a-8ad9-a2d5f06964a9/logo', 'rename': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/f881b5bc-5f27-434a-8ad9-a2d5f06964a9/rename', 'members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/f881b5bc-5f27-434a-8ad9-a2d5f06964a9/members', 'public_members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/com

[{'links': {'self': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e0cbc1d1-eea8-49bb-bab6-fd3227adffe1',
   'self_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/biostatistics-collaboration-center-lecture-series',
   'settings_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/biostatistics-collaboration-center-lecture-series/settings',
   'logo': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e0cbc1d1-eea8-49bb-bab6-fd3227adffe1/logo',
   'rename': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e0cbc1d1-eea8-49bb-bab6-fd3227adffe1/rename',
   'members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e0cbc1d1-eea8-49bb-bab6-fd3227adffe1/members',
   'public_members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e0cbc1d1-eea8-49bb-bab6-fd3227adffe1/members/public',
   'invitations': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/e0cbc1d1-eea8-49bb-bab6-fd3227adffe1

In [113]:
## The function "make_community_df" takes in the "multiple_slug_list" and uses json_normalize to
#flatten the json contained in the "hits" field. The function returns a dataframe called the "digitalhub_community_df".

def make_community_df(multiple_slug_list):
    #final_list = json_normalize(multiple_author_list, meta=["search-results"], record_path=["search-results", "entry"])
    #digitalhub_community_df = pd.DataFrame.from_dict(json_normalize(multiple_slug_list, record_path=["hits", "hits"]),orient="columns")
    digitalhub_community_df = pd.DataFrame.from_dict(json_normalize(multiple_slug_list, max_level = 3), orient='columns')
    return digitalhub_community_df

digitalhub_nested_community_df = make_community_df(multiple_slug_list)
make_community_df(multiple_slug_list)

#References
#https://stackoverflow.com/questions/48177934/flatten-or-unpack-list-of-nested-dicts-in-dataframe
#https://stackoverflow.com/questions/50161070/convert-list-of-dicts-of-dict-into-dataframe
#https://stackoverflow.com/questions/43984865/python-having-trouble-returning-a-pandas-data-frame-from-a-user-defined-functio
#https://stackoverflow.com/questions/37668291/flatten-double-nested-json

C:\Users\keg827\AppData\Local\Temp\ipykernel_30316\444227379.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  digitalhub_community_df = pd.DataFrame.from_dict(json_normalize(multiple_slug_list, max_level = 3), orient='columns')
C:\Users\keg827\AppData\Local\Temp\ipykernel_30316\444227379.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  digitalhub_community_df = pd.DataFrame.from_dict(json_normalize(multiple_slug_list, max_level = 3), orient='columns')


,created,slug,updated,revision_id,id,links.self,links.self_html,links.settings_html,links.logo,links.rename,...,links.invitations,links.requests,metadata.type.id,metadata.type.title.en,metadata.website,metadata.description,metadata.title,access.record_policy,access.visibility,access.member_policy
0,2022-07-19T14:57:37.082943+00:00,biostatistics-collaboration-center-lecture-series,2022-07-19T14:57:37.309092+00:00,2,e0cbc1d1-eea8-49bb-bab6-fd3227adffe1,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,project,Project,http://www.feinberg.northwestern.edu/sites/bcc...,Free introductory lectures on biostatistics in...,Biostatistics Collaboration Center Lecture Series,open,public,open
1,2022-07-19T14:57:37.561025+00:00,center-for-community-health,2022-07-19T14:57:37.614767+00:00,2,bda2571b-a615-4f62-8cf7-08df4d16b200,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,organization,Organization,https://www.feinberg.northwestern.edu/sites/cch/,NaN,Center for Community Health,open,public,open
2,2022-07-19T14:57:37.750182+00:00,center-for-biomedical-informatics-and-data-sci...,2022-07-19T14:57:37.799752+00:00,2,714c1318-2f3e-46d6-b186-142baedd90f9,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,organization,Organization,https://www.feinberg.northwestern.edu/sites/au...,The Center for Data Science and Informatics (C...,Center for Biomedical Informatics and Data Sci...,open,public,open
3,2022-07-19T14:57:37.948710+00:00,dialogues-in-oncofertility,2022-07-19T14:57:38.000529+00:00,2,e77128e8-07bf-4acc-914e-221aac97173a,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,project,Project,https://oncofertility.msu.edu/,Dialogues in Oncofertility is a record of scho...,Dialogues in Oncofertility,open,public,open
4,2022-07-19T14:57:38.127791+00:00,galter-health-sciences-library-and-learning-ce...,2022-07-19T14:57:38.189236+00:00,2,df884fce-a826-4e57-9d6f-33b092757fa3,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,organization,Organization,https://galter.northwestern.edu,The Galter Health Sciences Library & Learning ...,Galter Health Sciences Library and Learning Ce...,open,public,open
5,2022-07-19T14:57:38.292695+00:00,masters-in-public-health-ce-papers,2022-07-19T14:57:38.353122+00:00,2,cf57fb15-f867-4546-b46f-0838c41f0bf3,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northw

In [ ]:
#The function "flatten_search_results" takes in the "scopus_articles_df" and flattens the json contained in the
#"search results" field and creates series then columns of the content. The funciton returns a pandas dataframe
#called "remove_opensearchQuery_nest" which is renamed "scopus_flattened_search_results_df". 

# def flatten_search_results(digitalhub_nested_community_df):
#     remove_hits_nest = pd.concat([digitalhub_nested_community_df.drop(['hits'], axis=1), digitalhub_nested_community_df['hits'].apply(pd.Series)], axis=1, join="outer")
#     remove_aggregations_nest = pd.concat([remove_hits_nest.drop(['aggregations'], axis=1), remove_hits_nest['aggregations'].apply(pd.Series)], axis=1, join="outer")
#     remove_links_nest = pd.concat([remove_aggregations_nest.drop(['links'], axis=1), remove_aggregations_nest['links'].apply(pd.Series)], axis=1, join="outer")
#     return remove_links_nest

# digitalhub_flattened_community = flatten_search_results(digitalhub_nested_community_df)
# flatten_search_results(digitalhub_community_df)

#References
#https://stackoverflow.com/questions/29325458/dictionary-column-in-pandas-dataframe/29330853

In [ ]:
# def flatten_search_results_2(digitalhub_flattened_community):
#     remove_hits_nest = pd.concat([digitalhub_flattened_community.drop(['hits'], axis=1), digitalhub_flattened_community['hits'].apply(pd.Series)], axis=1, join="outer")
#     remove_type_nest = pd.concat([remove_hits_nest.drop(['type'], axis=1), remove_hits_nest['type'].apply(pd.Series)], axis=1, join="outer")
#     remove_visibility_nest = pd.concat([remove_type_nest.drop(['visibility'], axis=1), remove_type_nest['visibility'].apply(pd.Series)], axis=1, join="outer")
#     return remove_visibility_nest

# digitalhub_flattened_community_2 = flatten_search_results_2(digitalhub_flattened_community)
# flatten_search_results_2(digitalhub_flattened_community)

In [114]:
## Export file to excel, without the Pandas index, but with the header

digitalhub_nested_community_df.to_excel("outputs/prism_communities.xlsx", index=False, header=True)

In [ ]:
## Load Prism Communities JSON

with open('data\prism communities.json') as f:
    prism_json_data = json.load(f)   

    print(prism_json_data)


In [ ]:
## Create a dataframe from Prism Communities JSON
prism_json_communities_df = pd.DataFrame.from_dict(json_normalize(prism_json_data, max_level=2))
prism_json_communities_df.head(10)

In [ ]:
## Export file to excel, without the Pandas index, but with the header
prism_json_communities_df.to_excel("outputs/prism_communities.xlsx", index=False, header=True)

In [ ]:
# import json_schema
# from json_schema import json_schema
## https://pypi.org/project/json-schema-matcher/
## https://github.com/nano-labs/json_schema

In [ ]:
# from types import SimpleNamespace

# print(type(digitalhub_json_data[1]))
# print(type(prism_json_data[1]))

# digitalhub_json_data_json = json.dumps(digitalhub_json_data[1])
# print(digitalhub_json_data_json)
# print("----------------------------------------------------")
# prism_json_data_json = json.dumps(prism_json_data[1])
# print(prism_json_data_json)

# # creating the object

# x = json.loads(digitalhub_json_data_json, object_hook = lambda d : SimpleNamespace(**d))
# y = json.loads(prism_json_data_json, object_hook = lambda d : SimpleNamespace(**d))

# print(x.community.access)
# print("---------------------")
# print(y.community.links)


In [ ]:
# import time
# from selenium import webdriver
# driver = webdriver.Chrome(r'C:\Users\keg827\AppData\Local\ChromeDriver\chromedriver')  # Optional argument, if not specified will search path.
# driver.get('http://www.google.com/');
# time.sleep(5) # Let the user actually see something!
# search_box = driver.find_element_by_name('q')
# search_box.send_keys('ChromeDriver')
# search_box.submit()
# time.sleep(5) # Let the user actually see something!
# driver.quit()

### Get Collections Data from DigitalHub

In [2]:
## Upload a .csv DigitalHub Collections that will become Prism Communities

digitalhub_community_path = "data/2022_08-01 DigitalHub Collection Migration Plan_Communities Only.csv"

## Read the CSV file and store into Pandas DataFrame 
digitalhub_community_df = pd.read_csv(digitalhub_community_path , encoding = "ISO-8859-1", na_values=['NULL', '<NA>'])

## encoding = "ISO-8859-1", na_values=['NULL', '<NA>']

#Change the column names to lower case with underscore for spaces
digitalhub_community_df.columns =  digitalhub_community_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")
digitalhub_community_df.head()

## Checked: No problems

C:\Users\keg827\AppData\Local\Temp\ipykernel_30316\4210224928.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  digitalhub_community_df.columns =  digitalhub_community_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")
C:\Users\keg827\AppData\Local\Temp\ipykernel_30316\4210224928.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  digitalhub_community_df.columns =  digitalhub_community_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")


,ï»¿description,community,access:_visibility,access:_member_policy,access:_record_policy,access:_owned_by,access:_reader,id,metadata:_title,metadata:_description,metadata:_type,metadata:_website,dh_id,number_of_collections,collection_name,child-collection_name_not_present_in_prism,number_of_items,item-level_id,item-level_doi,unnamed:_19
0,Community,Biostatistics Collaboration Center Lecture Series,Public,Open,Open,"akl5916, ccb638, tlm488",NaN,biostatistics-collaboration-center-lecture-series,Biostatistics Collaboration Center Lecture Series,Free introductory lectures on biostatistics in...,project,http://www.feinberg.northwestern.edu/sites/bc...,2cc92425-b656-47ea-a3b4-825405ee6088,6,NaN,NaN,22,NaN,NaN,NaN
1,Community,Center for Community Health,Public,Open,Open,"kah2923, gmr244",NaN,center-for-community-health,Center for Community Health,NaN,organization,https://www.feinberg.northwestern.edu/sites/cch/,ae0b945c-d0d4-45bb-a0fc-263c7afca49e,0,NaN,NaN,2,NaN,NaN,NaN
2,Community,Center for Biomedical Informatics and Data Sci...,Public,Open,Open,"jbs642, aco454, nds616, ssv0467, skh8463",NaN,center-for-biomedical-informatics-and-data-sci...,Center for Biomedical Informatics and Data Sci...,The Center for Data Science and Informatics (C...,organization,https://www.feinberg.northwestern.edu/sites/au...,2828d181-5de7-4567-ba8d-d6bdb72b625f,2,NaN,NaN,23,NaN,NaN,NaN
3,Community,Dialogues in Oncofertility,Public,Open,Open,"lma467, bkm413",NaN,dialogues-in-oncofertility,Dialogues in Oncofertility,Dialogues in Oncofertility is a record of scho...,project,https://oncofertility.msu.edu/,44857499-5126-47bd-a226-5fbf291abaf6,0,NaN,NaN,7,NaN,NaN,NaN
4,Community,Galter Health Sciences Library & Learning Center,Public,Open,Open,"pls360, kal660, keg827",NaN,galter-health-sciences-library-&-learning-center,Galter Health Sciences Library & Learning Center,The Galter Health Sciences Library & Learning ...,organization,https://galter.northwestern.edu/,fj2362114,10,NaN,NaN,68,NaN,NaN,NaN


In [3]:
## Extract a the Series for "dh_id" and transform into a list

digitalhub_community_series = digitalhub_community_df["dh_id"]
digitalhub_community_list = digitalhub_community_series.tolist()

print(digitalhub_community_list)

## Checked: No problems

['2cc92425-b656-47ea-a3b4-825405ee6088', 'ae0b945c-d0d4-45bb-a0fc-263c7afca49e', '2828d181-5de7-4567-ba8d-d6bdb72b625f', '44857499-5126-47bd-a226-5fbf291abaf6', 'fj2362114', '40acd700-b850-4e7b-a650-0535de84ab6b', nan, 'b0375b45-0b95-4bf8-9ee7-7df4d6fb47e4', 'hq37vn62g', '2e61510b-939f-4637-9fd9-c31f1013c661', '6682x398c', '61e54c3e-b36f-48de-89d7-1c59048e5e03', 'cfda59b0-7d3c-4aa7-9f5e-53665849d624', 'e71c88e3-3a0b-4b56-90ce-8ddc75a78e81', 'qr46r0806', '97a2913d-45b8-458c-82eb-5111a94b6c9f', 'pedneurbriefs', '93c81706-47f8-49a1-88a2-3acbc971f4ed', 'a4de96c9-7c6d-40d6-ad9e-cac8a24faad5', 'f2bf6e1d-0e32-4ce2-a52e-bb0522d5708d', 'ce39f2be-9a64-4717-973b-ff531c2a93ee', 'ed35d953-d035-44a7-bacb-0edf1c8006a6', '07b25bee-4a47-466a-b9b8-70d7a392fab0', nan, nan, '074eef38-f1fa-48bb-aa1a-dd895b73d2b1', '98aba013-3005-4cbe-9fb0-e21960a8e274', '27114af7-6444-4f3c-8de1-3e9c5dc73e95', 'a75049ad-e87e-4fb8-80f5-f6a5c6276889', '7b21d655-90c0-475e-814f-de7cb124fc18', 'f2a9d478-3411-412c-a88d-6de5b454c4

In [4]:
## Loop through list of DigitalHub Collection URLS (i.e. Communities) and use the urllib.request to get the json data
## test_list = ['2cc92425-b656-47ea-a3b4-825405ee6088', 'a86e1412-d72c-4cae-b8ca-16fd834cb128','fc389d13-2430-409b-82fd-a4b26613d350']

multi_digitalhub_community_list = []
digitalhub_community_problem_list = []

for item in digitalhub_community_list:
    try:

        with urllib.request.urlopen(f"https://digitalhub.northwestern.edu/collections/{item}.json" ) as url:
            single_digitalhub_community_dict = json.loads(url.read().decode())
            multi_digitalhub_community_list.append(single_digitalhub_community_dict)
            print(item)
        time.sleep(1)
       

    except urllib.error.HTTPError as http_err:
        print(item)
        digitalhub_community_problem_list.append(item)
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
        

    except urllib.error.URLError as url_err:
        print(item)
        digitalhub_community_problem_list.append(item)
        print(f'URL error occurred: {url_err}. ', 'Exiting the loop!')  # Python 3.6
       

    except json.JSONDecodeError as json_err:
        print(item)
        digitalhub_community_problem_list.append(item)
        print(f'JSON Decode error occurred: {json_err}. ', 'Poorly formed JSON.')  # Python 3.6
       
        
    except Exception as err:
        print(item)
        digitalhub_community_problem_list.append(item)
        print(f'Other error occurred: {err}. ')  # Python 3.6
        
        
       
    else:
        print('Success!')



## Resources
## https://docs.python.org/3/library/urllib.request.html

## Checked: No problems

2cc92425-b656-47ea-a3b4-825405ee6088
Success!
ae0b945c-d0d4-45bb-a0fc-263c7afca49e
Success!
2828d181-5de7-4567-ba8d-d6bdb72b625f
Success!
44857499-5126-47bd-a226-5fbf291abaf6
Success!
fj2362114
Success!
40acd700-b850-4e7b-a650-0535de84ab6b
Success!
nan
HTTP error occurred: HTTP Error 500: Internal Server Error
b0375b45-0b95-4bf8-9ee7-7df4d6fb47e4
Success!
hq37vn62g
Success!
2e61510b-939f-4637-9fd9-c31f1013c661
Success!
6682x398c
Success!
61e54c3e-b36f-48de-89d7-1c59048e5e03
Success!
cfda59b0-7d3c-4aa7-9f5e-53665849d624
JSON Decode error occurred: Expecting value: line 1 column 2 (char 1).  Poorly formed JSON.
e71c88e3-3a0b-4b56-90ce-8ddc75a78e81
Success!
qr46r0806
Success!
97a2913d-45b8-458c-82eb-5111a94b6c9f
JSON Decode error occurred: Expecting value: line 1 column 2 (char 1).  Poorly formed JSON.
pedneurbriefs
Success!
93c81706-47f8-49a1-88a2-3acbc971f4ed
Success!
a4de96c9-7c6d-40d6-ad9e-cac8a24faad5
Success!
f2bf6e1d-0e32-4ce2-a52e-bb0522d5708d
JSON Decode error occurred: Expecting

In [5]:
## Inspect the results of the URL Query for DigitalHub Collections that will become communities in Prism

# print(multi_digitalhub_community_list)
print(digitalhub_community_problem_list)

## Checked: No problems

[nan, 'cfda59b0-7d3c-4aa7-9f5e-53665849d624', '97a2913d-45b8-458c-82eb-5111a94b6c9f', 'f2bf6e1d-0e32-4ce2-a52e-bb0522d5708d', 'ce39f2be-9a64-4717-973b-ff531c2a93ee', nan, nan]


In [6]:
## Create a dataframe from DigitalHub json for DigitalHub Collections that will become communities in Prism

digitalhub_community_df = pd.DataFrame.from_dict(json_normalize(multi_digitalhub_community_list, max_level=1))
# digitalhub_community_df.head(10)

## Checked: No problems

C:\Users\keg827\AppData\Local\Temp\ipykernel_30316\319419172.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  digitalhub_community_df = pd.DataFrame.from_dict(json_normalize(multi_digitalhub_community_list, max_level=1))


In [7]:
## Upload .txt files of the problem Community json metadata

northwesternelements_path = r"data\community\northwesternelements.txt"
communicationbridge_path = r"data\community\communicationbridge.txt"
nucatsgrantsrepository_path = r"data\community\nucatsgrantsrepository.txt"
preventionmethodology_path = r"data\community\preventionmethodology.txt"

problem_dict_list = []

path_list = [northwesternelements_path, communicationbridge_path,nucatsgrantsrepository_path, preventionmethodology_path]
for path in path_list: 
#     print(path)
    with open(path) as f:
        problem_dict = json.load(f)
        problem_dict_list.append(problem_dict)
    
# print(problem_dict_list)

## Checked: No problems

In [8]:
problem_df = pd.DataFrame.from_dict(json_normalize(problem_dict_list, max_level=1))
# problem_df.head()

## Checked: No problems

C:\Users\keg827\AppData\Local\Temp\ipykernel_30316\786300000.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  problem_df = pd.DataFrame.from_dict(json_normalize(problem_dict_list, max_level=1))


In [9]:
## Concatenate the DigitalHub Community Dataframe to the problem_df

digitalhub_community_df = pd.concat([digitalhub_community_df, problem_df], axis=0)
digitalhub_community_df.reset_index(inplace=True, drop=True) 
# digitalhub_community_df.head(10)

## Checked: No problems

In [10]:
#### NOT NEEDED FOR RUN THROUGH DIGITALHUB, just for adding to "desired" organization for migrating to Prism

## Create some communities

added_communities_df = pd.DataFrame([{'Multi-page?':[], 
                                      'Title':'Galter Library Audio-Video Archives',
                                      'Keyword':[], 
                                      'Resource type(s)':[], 
                                      'Rights':[],
                                      'Creator':[], 
                                      'Contributor':[], 
                                      'Description':[], 
                                      'Abstract':[],
                                      'Original Bibliographic Citation':[], 
                                      'Related URL':[], 
                                      'Publisher':[],
                                      'Date Created':[], 
                                      'Original Identifier':[], 
                                      'Language':[], 
                                      'Subject: MESH':[],
                                      'Subject: LCSH':[], 
                                      'Subject: Geographic Name':[], 
                                      'Subject: Name':[],
                                      'Location':[],
                                      'Digital Origin':[], 
                                      'Id':'galter-library-audio-video-archives',                                           
                                      'uri':[],
                                      'members':[{'Id':'cece380d-4dee-4e4e-aa97-28cb1d4f6b19'},
                                                  {'Id':'student-life'},
                                                  {'Id':'paul-de-kruif-interviews'},
                                                  {'Id':'b1546649-c60e-441b-9042-ec7a27adaf66'}]                                                 
                                     },
                                     {'Multi-page?':[], 
                                      'Title':'History of Feinberg School of Medicine',                                               
                                      'Keyword':[], 
                                      'Resource type(s)':[], 
                                      'Rights':[],
                                      'Creator':[], 
                                      'Contributor':[], 
                                      'Description':[], 
                                      'Abstract':[],
                                      'Original Bibliographic Citation':[], 
                                      'Related URL':[], 
                                      'Publisher':[],
                                      'Date Created':[], 
                                      'Original Identifier':[], 
                                      'Language':[], 
                                      'Subject: MESH':[],
                                      'Subject: LCSH':[], 
                                      'Subject: Geographic Name':[], 
                                      'Subject: Name':[],
                                      'Location':[],
                                      'Digital Origin':[], 
                                      'Id':'history-of-feinberg-school-of-medicine',                                                                            
                                      'uri':[],
                                      'members':[{'Id':'2f75r807r'},
                                                  {'Id':'areybook'},
                                                  {'Id':'5712m6524'},
                                                  {'Id': '6d56zw644'},
                                                  {'Id': '01d4023f-da66-45c4-8987-af7badda959a'},
                                                  {'Id': 'h702q639b'},
                                                  {'Id': '47429913s'},
                                                  {'Id': '6m311p28w'},
                                                  {'Id': 'pr76f340k'},
                                                  {'Id': 'f670ed27-d344-4cb1-aa34-e3339f2992d5'},
                                                  {'Id': 'aa85a365-c493-42cf-922c-21ab24407a1e'},
                                                  {'Id': 'c85b38a8-e367-4ef4-8c0a-35a94c479dc6'},
                                                  {'Id': 'e13451f1-01ff-4462-acbf-2a87eb2312e2'},
                                                  {'Id': '3f4ac3b8-6c05-4b4d-978e-f5ab7388743b'},
                                                  {'Id': '7d8b5c55-9d80-40a5-992f-7bbadd466d5b'},
                                                  {'Id': '506cbdca-cd9d-43fd-9be7-4b1b912782e5'},
                                                  {'Id': '695a2e0f-428f-41ea-acd8-66486a1e292b'},
                                                  {'Id': 'ae887507-5e30-4129-91c2-dda6f8e5f944'},
                                                  {'Id': '9cc06089-1a82-4378-ad19-b93dbbd402cb'},
                                                  {'Id': '9f47bff1-33b4-4d63-88b2-75aab10b84bb'},
                                                  {'Id': '0a5c7f42-7b10-443e-985a-b773a04e15ba'},
                                                  {'Id': 'd520d291-a3ab-431c-a992-6a931779ff31'},
                                                  {'Id': 'b6971153-c8bd-4f03-9917-d68659a89784'},
                                                  {'Id': '752b7c7f-ec96-4635-af07-34d4b36dac78'},
                                                  {'Id': '3f4ac3b8-6c05-4b4d-978e-f5ab7388743b'},
                                                  {'Id': '9k41zd48h'}]
                                     },
                                     { 'Multi-page?':[], 
                                      'Title':'Researchers Collections',
                                      'Keyword':[], 
                                      'Resource type(s)':[], 
                                      'Rights':[],
                                      'Creator':[], 
                                      'Contributor':[], 
                                      'Description':[], 
                                      'Abstract':[],
                                      'Original Bibliographic Citation':[], 
                                      'Related URL':[], 
                                      'Publisher':[],
                                      'Date Created':[], 
                                      'Original Identifier':[], 
                                      'Language':[], 
                                      'Subject: MESH':[],
                                      'Subject: LCSH':[], 
                                      'Subject: Geographic Name':[], 
                                      'Subject: Name':[],
                                      'Location':[],
                                      'Digital Origin':[], 
                                      'Id':'researchers-collections',                                           
                                      'uri':[],
                                      'members':[{'Id': 'e1683f-5075-4afd-9eba-2a36fc981414'},
                                                  {'Id': '7badb7c9-d4ec-4ca9-b58e-6e01f224fcf7'},
                                                  {'Id': 'd0798568-47c3-453e-ae39-242d8a96b1dc'},
                                                  {'Id': '913f8fa2-06c9-49e1-9cdf-0f88118b18da'},
                                                  {'Id': '8s45q876k'},
                                                  {'Id': '3ca02e5e-83be-4ea7-b51f-11aee3497e6c'},
                                                  {'Id': 'afec3d3f-5ee6-468a-b8b4-80ab6d0402ac'},
                                                  {'Id': 'ea926798-0e47-4441-b159-8af916499af3'},
                                                  {'Id': '6b2bc47e-a3da-4222-8b8b-39e3b2832648'},
                                                  {'Id': 'fj236212d'},
                                                  {'Id': 'ed5f344a-8a48-4b50-9843-895634e5cd6a'},
                                                  {'Id': 'kw52j804p'},
                                                  {'Id': '96fc0e70-98e7-4e49-9784-8aa6942fc2a6'},
                                                  {'Id': 'ae47e062-d7f8-49c0-8ffa-ce86fb2855ca'},
                                                  {'Id': '91a4a3c2-e9f8-4540-85bb-5b69923106c0'},
                                                  {'Id': '09e7110d-7677-4b82-985a-a7c26ac46b57'},
                                                  {'Id': '55da1441-ddee-4f57-9c17-d371f78f2ed4'},
                                                  {'Id': '9aa727f0-29d0-44af-ae9f-1018208cec89'},
                                                  {'Id': 'd50c6f56-2600-4e67-ba4f-ee681eeae64c'},
                                                  {'Id': '19673087-b6a5-4108-b285-9614aa8b6b95'},
                                                  {'Id': 'rb68xb84x'},
                                                  {'Id': '0d944080-d8ec-4386-abb0-c5ca34d2a3f5'},
                                                  {'Id': 'a0deab15-7c16-4c52-86f8-80c96a2fb888'},
                                                  {'Id': 'e0338411-7829-49ac-8fdc-cd17b7307474'},
                                                  {'Id': '2dd287f0-9748-41f7-8fab-222db450d196'},
                                                  {'Id': '5f1b1739-512f-4015-98bc-22f37f42af7b'},
                                                  {'Id': '96fc0e70-98e7-4e49-9784-8aa6942fc2a6'},
                                                  {'Id': '2d8b503f-203c-48ce-ac34-d0c976997761'},
                                                  {'Id': 'aa31fe0c-41ec-46fd-82c0-405f168a5606'},
                                                  {'Id': '1d4cede9-d8d6-4576-994d-91d36bd15b0b'},
                                                  {'Id': '74d22173-8b26-4d34-b0a0-8b7b15bca6f8'}]
                                     }])
                                      
                                           
added_communities_df.head()                           
## Checked: No problems  
   

,Multi-page?,Title,Keyword,Resource type(s),Rights,Creator,Contributor,Description,Abstract,Original Bibliographic Citation,...,Language,Subject: MESH,Subject: LCSH,Subject: Geographic Name,Subject: Name,Location,Digital Origin,Id,uri,members
0,[],Galter Library Audio-Video Archives,[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],galter-library-audio-video-archives,[],[{'Id': 'cece380d-4dee-4e4e-aa97-28cb1d4f6b19'...
1,[],History of Feinberg School of Medicine,[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],history-of-feinberg-school-of-medicine,[],"[{'Id': '2f75r807r'}, {'Id': 'areybook'}, {'Id..."
2,[],Researchers Collections,[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],researchers-collections,[],"[{'Id': 'e1683f-5075-4afd-9eba-2a36fc981414'},..."


In [11]:
#### NOT NEEDED FOR RUN THROUGH DIGITALHUB, just for adding to "desired" organization for migrating to Prism

## Concatenate the DigitalHub Community Dataframe to the added_communities_df

digitalhub_community_df = pd.concat([digitalhub_community_df, added_communities_df], axis=0)
digitalhub_community_df.reset_index(inplace=True, drop=True) 
# digitalhub_community_df.head(10)

## Checked: No problems

In [12]:
## Add Column to digitalhub_community_df to indicate that these results are a community

digitalhub_community_df["Level Type"] = "Community"
digitalhub_community_df["Level Number"] = "1"
digitalhub_community_df['Level Number'] = digitalhub_community_df['Level Number'].apply(int)

## Create a column from the index

digitalhub_community_df['community_rowid'] = digitalhub_community_df.index

## Create a new column called Community_ID

digitalhub_community_df['Community_ID'] = digitalhub_community_df['Id']

## Checked: No problems

In [13]:
## Inspect the "members" column and create a list of member IDs

row_member_list = []
column_member ={}
count_member = {}

for k, v in digitalhub_community_df["members"].items():
    for value in v: 
        member = value["Id"]
        row_member_list.append(member)
    column_member[k] = row_member_list
    count_member[k] = len(row_member_list)
    row_member_list =[]
    row_member_count = []

## Append the column member dictionary to the DigitalHub Community DF dataframe

digitalhub_community_df['Member_List'] = digitalhub_community_df.index.map(column_member)
digitalhub_community_df['Member_List_Count'] = digitalhub_community_df.index.map(count_member)
digitalhub_community_df.head()

## Checked: No problems

,Multi-page?,Title,Keyword,Resource type(s),Rights,Creator,Contributor,Description,Abstract,Original Bibliographic Citation,...,Digital Origin,Id,uri,members,Level Type,Level Number,community_rowid,Community_ID,Member_List,Member_List_Count
0,False,Biostatistics Collaboration Center Lecture Series,"[Medical research, Biostatistics]",[Presentation],[All rights reserved],[],[],Free introductory lectures on biostatistics in...,[],[],...,[],2cc92425-b656-47ea-a3b4-825405ee6088,https://digitalhub.northwestern.edu/collection...,"[{'Multi-page?': True, 'Title': ['2019-2020'],...",Community,1,0,2cc92425-b656-47ea-a3b4-825405ee6088,"[a86e1412-d72c-4cae-b8ca-16fd834cb128, fc389d1...",4
1,False,Center for Community Health,"[Evaluation, Data Collection, CCH, Stakeholder...",[],[],"[Herzog, Keith Alan, Robles-Schrader, Grisel M...","[Robles-Schrader, Grisel Marie]",,[],[],...,[],ae0b945c-d0d4-45bb-a0fc-263c7afca49e,https://digitalhub.northwestern.edu/collection...,"[{'Title': ['CCH Evaluation Metrics Table'], '...",Community,1,1,ae0b945c-d0d4-45bb-a0fc-263c7afca49e,"[e342fabd-2faa-42bf-bd39-582fdac49a0d, 4e85036...",3
2,False,Center for Biomedical Informatics and Data Sci...,"[CDSI, informatics, NUCATS, data science, CBID...",[],[],[],[],The Center for Data Science and Informatics (C...,[],[],...,[],2828d181-5de7-4567-ba8d-d6bdb72b625f,https://digitalhub.northwestern.edu/collection...,"[{'Multi-page?': False, 'Title': ['Biomedical ...",Community,1,2,2828d181-5de7-4567-ba8d-d6bdb72b625f,"[ab570598-e497-4c81-9351-1aa316252682, 6d66813...",2
3,False,Dialogues in Oncofertility,"[oncofertility, collaboration, communication s...",[],[All rights reserved],"[Ataman, Lauren]","[Ataman, Lauren, Woodruff, Teresa K, Smith, Br...",Dialogues in Oncofertility is a record of scho...,[],[],...,[],44857499-5126-47bd-a226-5fbf291abaf6,https://digitalhub.northwestern.edu/collection...,[{'Title': ['A SWOT Analysis of Oncofertility:...,Community,1,3,44857499-5126-47bd-a226-5fbf291abaf6,"[7f794595-230e-48a0-aee1-d92f567d09a7, 28201f5...",7
4,None,Galter Health Sciences Library Collections,[Galter Health Sciences Library],[],[],[],[],None,[],[],...,[],fj2362114,https://digitalhub.northwestern.edu/collection...,"[{'Multi-page?': False, 'Title': ['12th Genera...",Community,1,4,fj2362114,"[07b25bee-4a47-466a-b9b8-70d7a392fab0, diaw201...",6


In [14]:
#### NOT NEEDED FOR RUN THROUGH DIGITALHUB, just for adding to "desired" organization for migrating to Prism

## Add subcollections to GHSL: 
# 8c1d851c-a5e6-4790-a867-fa889e66630e - med subject headings

# 91294b2e-34e4-46ac-9086-be17c40d0d01 - operation saving lives
# 8a281ff6-dd0e-4a02-8486-97c3bc7058c4 - pursuit of a grand cause
# k0698748f - daniel hale
# d526f63d-f10a-423d-bb81-bc32cc70b427 - notable women
# ec202d45-992f-4f21-b3ea-d02703ca7621 - men behind

add_to_series = [{'Multi-page?':[], 
                  'Title':'Operation Saving Lives: Northwestern as the 12th General Hospital During WWI',
                  'Keyword':[], 
                  'Resource type(s)':[], 
                  'Rights':[],
                  'Creator':[], 
                  'Contributor':[], 
                  'Description':[], 
                  'Abstract':[],
                  'Original Bibliographic Citation':[], 
                  'Related URL':[], 
                  'Publisher':[],
                  'Date Created':[], 
                  'Original Identifier':[], 
                  'Language':[], 
                  'Subject: MESH':[],
                  'Subject: LCSH':[], 
                  'Subject: Geographic Name':[], 
                  'Subject: Name':[],
                  'Location':[],
                  'Digital Origin':[], 
                  'Id':'91294b2e-34e4-46ac-9086-be17c40d0d01',                                           
                  'uri':[],
                  'members':[{'Id':'ec5cbb80-61dd-47a4-87eb-c5bacd211e90'},
                              {'Id':'5185bcc8-bc82-4e9b-aeca-a6aec040a8eb'},
                              {'Id':'710ff4c3-8f1e-480d-baf9-a425ea936534'},
                              {'Id':'c1cacdba-9705-4420-b58b-02e9975cc02b'},
                              {'Id':'4f0d22bb-84f9-41aa-a834-a9fa1609d1ac'}]                                                 
                 },{'Multi-page?':[], 
                  'Title':'Medical Subject Headings-Library of Congress Subject Headings Mapping Data',
                  'Keyword':[], 
                  'Resource type(s)':[], 
                  'Rights':[],
                  'Creator':[], 
                  'Contributor':[], 
                  'Description':[], 
                  'Abstract':[],
                  'Original Bibliographic Citation':[], 
                  'Related URL':[], 
                  'Publisher':[],
                  'Date Created':[], 
                  'Original Identifier':[], 
                  'Language':[], 
                  'Subject: MESH':[],
                  'Subject: LCSH':[], 
                  'Subject: Geographic Name':[], 
                  'Subject: Name':[],
                  'Location':[],
                  'Digital Origin':[], 
                  'Id':'91294b2e-34e4-46ac-9086-be17c40d0d01s',                                           
                  'uri':[],
                  'members':[{'Id':'3e59c5b9-bbbf-4f49-946f-e08ef9b10d9f'},
                              {'Id':'4b7f6a77-9cf4-4deb-b9aa-f3f49391bcc8'}] 

                 },{'Multi-page?':[], 
                  'Title':'In Pursuit of a Grand Cause',
                  'Keyword':[], 
                  'Resource type(s)':[], 
                  'Rights':[],
                  'Creator':[], 
                  'Contributor':[], 
                  'Description':[], 
                  'Abstract':[],
                  'Original Bibliographic Citation':[], 
                  'Related URL':[], 
                  'Publisher':[],
                  'Date Created':[], 
                  'Original Identifier':[], 
                  'Language':[], 
                  'Subject: MESH':[],
                  'Subject: LCSH':[], 
                  'Subject: Geographic Name':[], 
                  'Subject: Name':[],
                  'Location':[],
                  'Digital Origin':[], 
                  'Id':'8a281ff6-dd0e-4a02-8486-97c3bc7058c4',                                           
                  'uri':[],
                  'members':[{'Id':'f9da4c66-fab8-4ecf-9862-8b016ed0a124'},
                             {'Id':'f4a757f6-a617-496b-b040-df0b27ff8cf2'},
                             {'Id':'20dcde30-f1f6-4386-9620-6fa5b1cb3467'},
                             {'Id':'66708bc0-e8d0-4b44-8057-c0b8dbaa0566'},
                             {'Id':'2320576d-a6c8-422f-a074-b34180d15442'},
                             {'Id':'33e474af-6e46-4bac-9e42-96dbf4bcb51e'}]
                },{'Multi-page?':[], 
                  'Title':'Daniel Hale Williams, Surgeon, Educator & Medical Advocate',
                  'Keyword':[], 
                  'Resource type(s)':[], 
                  'Rights':[],
                  'Creator':[], 
                  'Contributor':[], 
                  'Description':[], 
                  'Abstract':[],
                  'Original Bibliographic Citation':[], 
                  'Related URL':[], 
                  'Publisher':[],
                  'Date Created':[], 
                  'Original Identifier':[], 
                  'Language':[], 
                  'Subject: MESH':[],
                  'Subject: LCSH':[], 
                  'Subject: Geographic Name':[], 
                  'Subject: Name':[],
                  'Location':[],
                  'Digital Origin':[], 
                  'Id':'k0698748f',                                           
                  'uri':[],
                  'members':[]
                },{'Multi-page?':[], 
                  'Title':'Notable Women of the Womans Medical School',
                  'Keyword':[], 
                  'Resource type(s)':[], 
                  'Rights':[],
                  'Creator':[], 
                  'Contributor':[], 
                  'Description':[], 
                  'Abstract':[],
                  'Original Bibliographic Citation':[], 
                  'Related URL':[], 
                  'Publisher':[],
                  'Date Created':[], 
                  'Original Identifier':[], 
                  'Language':[], 
                  'Subject: MESH':[],
                  'Subject: LCSH':[], 
                  'Subject: Geographic Name':[], 
                  'Subject: Name':[],
                  'Location':[],
                  'Digital Origin':[], 
                  'Id':'d526f63d-f10a-423d-bb81-bc32cc70b427',                                           
                  'uri':[],
                  'members':[]
                },{'Multi-page?':[], 
                  'Title':'Men Behind the Women at the Womans Medical School at Northwestern University Medical School',
                  'Keyword':[], 
                  'Resource type(s)':[], 
                  'Rights':[],
                  'Creator':[], 
                  'Contributor':[], 
                  'Description':[], 
                  'Abstract':[],
                  'Original Bibliographic Citation':[], 
                  'Related URL':[], 
                  'Publisher':[],
                  'Date Created':[], 
                  'Original Identifier':[], 
                  'Language':[], 
                  'Subject: MESH':[],
                  'Subject: LCSH':[], 
                  'Subject: Geographic Name':[], 
                  'Subject: Name':[],
                  'Location':[],
                  'Digital Origin':[], 
                  'Id':'ec202d45-992f-4f21-b3ea-d02703ca7621',                                           
                  'uri':[],
                  'members':[]
                  }] 

current_list=[]

## Add new subcollections to GHSL

current_series = digitalhub_community_df.loc[digitalhub_community_df['Id'] == 'fj2362114','members']
# print(current_series.item()[0])
current_list = current_series.tolist()
result = []
result = current_series.item() + [x for x in add_to_series if x not in current_series.items()]
print(len(result))


## Remove subcollections from GHSL

#x633f100h
#v405s9425

print(len(result))
for i in range(len(result)):
#     print(i)
    if result[i]['Id'] == 'v405s9425':
        print("True")
        del result[i]
        break


print(len(result))
for i in range(len(result)):
#     print(i)
    if result[i]['Id'] == 'x633f100h':
        print("True")
        del result[i]
        break



## Add result back to member

digitalhub_community_df['members'] = digitalhub_community_df['members'].astype('object')
digitalhub_community_df.at[4,'members'] = result

## Resources
## https://stackoverflow.com/questions/26483254/python-pandas-insert-list-into-a-cell
## https://www.geeksforgeeks.org/python-removing-dictionary-from-list-of-dictionaries/

12
12
True
11
True


In [15]:
## Export file to excel, with the Pandas index, and with the headers

digitalhub_community_df.to_excel("outputs/digitalhub_community_df.xlsx", header=True)

## Checked: No problems

In [16]:
## Create a dataframe for each members row and concatenate all of these into one digitalHub_sub_collection_df

members_dfs_list = []

for k, v in digitalhub_community_df["members"].items():
#     print("this is k: ", k)
#     print("this is v: ", v)
    member_df = pd.json_normalize(v)
    member_df['community_rowid'] = k
    members_dfs_list.append(member_df)

digitalhub_sub_collection_df = pd.concat(members_dfs_list, sort=False).reset_index(drop='index')
digitalhub_sub_collection_df.head()

## Resource
## https://stackoverflow.com/questions/62816027/convert-pandas-json-column-to-multiple-rows

## Checked: No problems

,Multi-page?,Title,Keyword,Resource type(s),Rights,Creator,Contributor,Description,Abstract,Original Bibliographic Citation,...,community_rowid,Page Number,Acknowledgments,Grants And Funding,DOI,ARK,File Size,File Format,download,members
0,True,[2019-2020],"[Biostatistics Collaboration Center, Statistic...",None,[All rights reserved],None,None,[Free introductory lectures on biostatistics i...,None,None,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,[Fall 2016],"[Biostatistics Collaboration Center , biostati...",None,[All rights reserved],None,None,[Free introductory lectures on biostatistics i...,None,None,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,[Fall 2017],"[medical research, Biostatistics Collaboration...",None,[All rights reserved],None,None,[Free introductory lectures on biostatistics i...,None,None,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,[Winter 2019],"[Biostatistics Collaboration Center , medical ...",None,[All rights reserved],None,None,[Free introductory lectures on biostatistics i...,None,None,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,[CCH Evaluation Metrics Table],"[Community Engagement Metrics, Evaluation]",[Resource Guides],[http://creativecommons.org/licenses/by/3.0/us/],"[Robles-Schrader, Grisel Marie, Herzog, Keith ...",None,[This table is a guide to the forms and survey...,None,None,...,1,None,None,None,[doi:10.18131/g3-8vsh-h698],None,374492.0,[pdf (Portable Document Format)],https://digitalhub.northwestern.edu/downloads/...,NaN


In [17]:
## Remove lists (i.e. brackets) and convert to strings for data in each column

# def list2Str(lst):
#     if type(lst) is list: # apply conversion to list columns
#         return";".join(lst)
#     else:
#         return str

# digitalhub_sub_collection_df.apply(lambda x: [list2Str(i) for i in x])

# digitalhub_sub_collection_df.head()

## Resources
## https://stackoverflow.com/questions/38147447/how-to-remove-square-bracket-from-pandas-dataframe


In [18]:
## Add Column to digitalhub_sub_collection_df to indicate that these results are a sub_collection or a record

# digitalhub_sub_collection_df["Level"] =  "Collection"

digitalhub_sub_collection_df['Level Type'] = np.where(digitalhub_sub_collection_df['DOI'].isnull(),"Sub_Collection", "Record" )
digitalhub_sub_collection_df['Level Number'] = np.where(digitalhub_sub_collection_df['DOI'].isnull(),"2", "6" )
digitalhub_sub_collection_df['Level Number'] = digitalhub_sub_collection_df['Level Number'].apply(int)

# digitalhub_sub_collection_df.dtypes

## Checked: No problems

In [19]:
## Export file to excel, without the Pandas index, but with the header
digitalhub_sub_collection_df.to_excel("outputs/digitalhub_sub_collection_df.xlsx", index=False, header=True)

## Checked: No problems

In [20]:
## Concatenate the digitalhub_community_df to the digitalhub_sub_collection_df using the 'community_rowid'
digitalhub_comm_col_df = pd.concat([digitalhub_community_df, digitalhub_sub_collection_df], axis=0).sort_values(by=['community_rowid'])
digitalhub_comm_col_df .reset_index(inplace=True, drop=True) 
# digitalhub_comm_col_df.head()

## Checked: No problems

In [21]:
## Sort the dataframe by rowid and Level Number
digitalhub_comm_col_df.sort_values(by = ['community_rowid','Level Number'], ascending = [True, True], inplace=True)

## Checked: No problems

In [22]:
# ## Fill forward the Community ID into subcollections and records
digitalhub_comm_col_df['Community_ID'] = digitalhub_comm_col_df.groupby(['community_rowid'])['Community_ID'].ffill()

## Resources
## https://stackoverflow.com/questions/64795941/how-do-i-forward-fill-nas-with-condition-of-2-other-cells-being-equal-in-pandas

## Checked: No problems

In [23]:
## Create a column for Sub_Collection ID
digitalhub_comm_col_df['Sub_Collection_ID'] = np.where(digitalhub_comm_col_df['Level Type'] == 'Sub_Collection',digitalhub_comm_col_df['Id'], np.nan)

## Resources: 
## https://stackoverflow.com/questions/67043249/how-to-use-np-where-in-creating-new-column-using-previous-rows

## Checked: No problems

In [24]:
## Export file to excel, without the Pandas index, but with the header
digitalhub_comm_col_df.to_excel("outputs/digitalhub_comm_col_df.xlsx", index=False, header=True)

## Checked: No problems

##### Recall Sub_Collection 

In [25]:
## Extract a the series for sub_collection 'ids' and transform into a list

digitalhub_sub_collection_series = digitalhub_comm_col_df[digitalhub_comm_col_df["Level Type"] == "Sub_Collection"]['Id']
digitalhub_sub_collection_list = digitalhub_sub_collection_series.tolist()

print(digitalhub_sub_collection_list)

## Checked: No problems

['17c76dde-4be6-4d2d-9020-3904a9451617', '5d8b5f8e-c46b-4e97-a5a2-9c118c9cca24', 'fc389d13-2430-409b-82fd-a4b26613d350', 'a86e1412-d72c-4cae-b8ca-16fd834cb128', '6d66813c-a6e8-49b0-a568-d9378f82da51', 'ab570598-e497-4c81-9351-1aa316252682', 'areybook', '91294b2e-34e4-46ac-9086-be17c40d0d01s', '91294b2e-34e4-46ac-9086-be17c40d0d01', 'd526f63d-f10a-423d-bb81-bc32cc70b427', 'library-notes', 'diaw2017', '07b25bee-4a47-466a-b9b8-70d7a392fab0', 'ec202d45-992f-4f21-b3ea-d02703ca7621', '8a281ff6-dd0e-4a02-8486-97c3bc7058c4', 'k0698748f', 'b0375b45-0b95-4bf8-9ee7-7df4d6fb47e4', 'pnb-1', 'pnb-2', 'pnb-4', 'pnb-14', 'pnb-5', 'pnb-6', 'pnb-7', 'pnb-8', 'pnb-9', 'pnb-10', 'pnb-11', 'pnb-12', 'pnb-3', 'pnb-13', 'pnb-16', 'pnb-17', 'pnb-29', 'pnb-28', 'pnb-26', 'pnb-25', 'pnb-24', 'pnb-27', 'pnb-22', 'pnb-21', 'pnb-20', 'pnb-19', 'pnb-18', 'pnb-15', 'pnb-23', 'f78501e9-ab3a-49db-ab85-d6de506988fc', '0b449311-b6c4-4144-843a-0ce84de567e2', 'e476cb5c-4200-421a-b8d3-a5262fee3643', '9e27fbd0-c6cb-47c7-877

In [26]:
## Loop through list of DigitalHub Sub_Collections and use the urllib.request to get the json data
## test_list = ['2cc92425-b656-47ea-a3b4-825405ee6088', 'a86e1412-d72c-4cae-b8ca-16fd834cb128','fc389d13-2430-409b-82fd-a4b26613d350']

multi_digitalhub_sub_collection_list = []
digitalhub_sub_collection_problem_list = []

for item in digitalhub_sub_collection_list:
    try:

        with urllib.request.urlopen(f"https://digitalhub.northwestern.edu/collections/{item}.json" ) as url:
            single_digitalhub_sub_collection_dict = json.loads(url.read().decode())
            multi_digitalhub_sub_collection_list.append(single_digitalhub_sub_collection_dict)
            print(item)
        time.sleep(1)
       

    except urllib.error.HTTPError as http_err:
        print(item)
        digitalhub_sub_collection_problem_list.append(item)
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
        

    except urllib.error.URLError as url_err:
        print(item)
        digitalhub_sub_collection_problem_list.append(item)
        print(f'URL error occurred: {url_err}. ', 'Exiting the loop!')  # Python 3.6
       

    except json.JSONDecodeError as json_err:
        print(item)
        digitalhub_sub_collection_problem_list.append(item)
        print(f'JSON Decode error occurred: {json_err}. ', 'Poorly formed JSON.')  # Python 3.6
       
        
    except Exception as err:
        print(item)
        digitalhub_sub_collection_problem_list.append(item)
        print(f'Other error occurred: {err}. ')  # Python 3.6
        
        
       
    else:
        print('Success!')



## Resources
## https://docs.python.org/3/library/urllib.request.html

## Checked: No problems

17c76dde-4be6-4d2d-9020-3904a9451617
Success!
5d8b5f8e-c46b-4e97-a5a2-9c118c9cca24
Success!
fc389d13-2430-409b-82fd-a4b26613d350
Success!
a86e1412-d72c-4cae-b8ca-16fd834cb128
Success!
6d66813c-a6e8-49b0-a568-d9378f82da51
Success!
ab570598-e497-4c81-9351-1aa316252682
Success!
areybook
Success!
91294b2e-34e4-46ac-9086-be17c40d0d01s
HTTP error occurred: HTTP Error 500: Internal Server Error
91294b2e-34e4-46ac-9086-be17c40d0d01
JSON Decode error occurred: Expecting value: line 1 column 2 (char 1).  Poorly formed JSON.
d526f63d-f10a-423d-bb81-bc32cc70b427
HTTP error occurred: HTTP Error 500: Internal Server Error
library-notes
Success!
diaw2017
Success!
07b25bee-4a47-466a-b9b8-70d7a392fab0
Success!
ec202d45-992f-4f21-b3ea-d02703ca7621
HTTP error occurred: HTTP Error 500: Internal Server Error
8a281ff6-dd0e-4a02-8486-97c3bc7058c4
Success!
k0698748f
HTTP error occurred: HTTP Error 500: Internal Server Error
b0375b45-0b95-4bf8-9ee7-7df4d6fb47e4
Success!
pnb-1
Success!
pnb-2
Success!
pnb-4
Succ

In [27]:
## Inspect the results of the URL Query for DigitalHub Collections that will become communities in Prism

# print(multi_digitalhub_sub_collection_list)
print(digitalhub_sub_collection_problem_list)

## Checked: No problems

['91294b2e-34e4-46ac-9086-be17c40d0d01s', '91294b2e-34e4-46ac-9086-be17c40d0d01', 'd526f63d-f10a-423d-bb81-bc32cc70b427', 'ec202d45-992f-4f21-b3ea-d02703ca7621', 'k0698748f', 'a76c9405-2fb7-47c0-bd6e-cd8cf9ea69a8', '2abe2c1b-932e-4bb0-8455-a97d828f6924', '1724d997-42e2-493d-88fd-22f836403628', '4dde1545-ad4e-4801-9db8-02d5aadc38c6', '113b8ab8-8f40-454b-9932-abd83e392f78', '6ce6bf89-1c23-4949-9ae3-1c6bd874e593', 'ef9651c9-c810-42e0-8e2d-58993586473d', 'eaa0acc4-64b5-4c34-8e90-49537ffa9e0b', '8c462e3c-4666-4997-9a29-4c17f7846c96', '22e29a25-6e89-4322-b772-15442ca5b713', '4a8457cb-7a65-4774-9ab6-057702e521f5', '779a5d4d-23b4-4a28-8c38-48faa43de46d', '01b2ba97-8603-477f-bde3-b36b6c79bdea', '691ddbbd-785c-4a6a-85d1-7569d8fcbcdb', '9aa9dbb6-7971-4e5c-9b9c-81216d9325d3', '1b2b7d0f-d7f3-41a4-a2a4-8435d719ecec', '46374bec-fe29-4a06-a8b8-f1f93668699c', '90c63af3-adb9-48dc-95d9-7b4bafe13a7a', 'b1546649-c60e-441b-9042-ec7a27adaf66', 'paul-de-kruif-interviews', 'student-life', '9k41zd48h', '3f4ac3b

In [28]:
## Create a dataframe called digitalhub_sub_collection_recall_df from DigitalHub json for DigitalHub Sub_Collections

digitalhub_sub_collection_recall_df = pd.DataFrame.from_dict(json_normalize(multi_digitalhub_sub_collection_list, max_level=1))
digitalhub_sub_collection_recall_df.head(10)

## Checked: No problems

C:\Users\keg827\AppData\Local\Temp\ipykernel_30316\3224943997.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  digitalhub_sub_collection_recall_df = pd.DataFrame.from_dict(json_normalize(multi_digitalhub_sub_collection_list, max_level=1))


,Multi-page?,Title,Keyword,Resource type(s),Rights,Creator,Contributor,Description,Abstract,Original Bibliographic Citation,...,Language,Subject: MESH,Subject: LCSH,Subject: Geographic Name,Subject: Name,Location,Digital Origin,Id,uri,members
0,False,Fall 2017,"[medical research, Biostatistics Collaboration...",[],[All rights reserved],[],[],Free introductory lectures on biostatistics in...,[],[],...,[],[],[],[],[],[],[],17c76dde-4be6-4d2d-9020-3904a9451617,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Biostatistics 101: the minor deta...
1,False,Winter 2019,"[Biostatistics Collaboration Center , medical ...",[],[All rights reserved],[],[],Free introductory lectures on biostatistics in...,[],[],...,[],[],[],[],[],[],[],5d8b5f8e-c46b-4e97-a5a2-9c118c9cca24,https://digitalhub.northwestern.edu/collection...,[{'Title': ['A Statistician's Guide to REDCap'...
2,False,Fall 2016,"[Biostatistics Collaboration Center , biostati...",[],[All rights reserved],[],[],Free introductory lectures on biostatistics in...,[],[],...,[],[],[],[],[],[],[],fc389d13-2430-409b-82fd-a4b26613d350,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Clinical Trials: Highlights from ...
3,True,2019-2020,"[Biostatistics Collaboration Center, Statistic...",[],[All rights reserved],[],[],Free introductory lectures on biostatistics in...,[],[],...,[],[Biostatistics],[],[],[],[],[],a86e1412-d72c-4cae-b8ca-16fd834cb128,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Logistic Regression: Odds & Ends'...
4,None,Chicago Biomedical Informatics and Data Scienc...,"[NUCATS, CDSI, informatics, data science]",[],[],[],[],Biomedical and Health Informatics and Data Sci...,[],[],...,[],[],[],[],[],[],[],6d66813c-a6e8-49b0-a568-d9378f82da51,https://digitalhub.northwestern.edu/collection...,[]
5,False,Biomedical Data Science Day,"[informatics, CDSI, data science, NUCATS, biom...",[],[],"[Soulakis, Nicholas Dean]",[],The Center for Data Science and Informatics pr...,[],[],...,[],[],[],[],[],[],[],ab570598-e497-4c81-9351-1aa316252682,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Breaking Data Silos: the Gen3 Pla...
6,True,Northwestern University Medical School 1859-1979,[History],[],[All rights reserved],"[Arey, Leslie Brainerd, 1891-]",[],"A history of the Feinberg School of Medicine, ...",[],[],...,[English],"[History, Schools, Medical]",[],[Chicago (Ill.)],"[Northwestern University (Evanston, Ill.). Med...",[],[],areybook,https://digitalhub.northwestern.edu/collection...,"[{'Title': ['Preface'], 'Resource type(s)': ['..."
7,False,Library Notes,"[Galter Health Sciences Library, Newsletter, A...",[],[All rights reserved],[Galter Health Sciences Library],[],Newsletter published by Galter Health Sciences...,[],[],...,[English],"[Chicago, Libraries, Medical]",[],[],[],[],[],library-notes,https://digitalhub.northwestern.edu/collection...,"[{'Title': ['Library Notes, April/May/June 199..."
8,None,"Dissemination, Impact, and Access Workshop 2017",[workshop],[],[],[],[],None,[],[],...,[],[],[],[],[],[],[],diaw2017,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Complying with the NIH Public Acc...
9,False,12th General Hospital Collection,[World War II],[],[],"[Connor, James A., Mason, Michael L., 1895-1963]",[],The 12th General Hospital Collection is compri...,[],[],...,[English],"[Hospitals, Military]","[World War, 1939-1945]","[Naples (Italy), Rome (Italy), Livorno (Italy)...",[12th General Hospital],[],[],07b25bee-4a47-466a-b9b8-70d7a392fab0,https://digitalhub.northwestern.edu/collection...,"[{'Multi-page?': False, 'Title': ['James A. Co..."


In [29]:
## Upload .txt files of the problem Sub_Collection json metadata

archive_path = r"data\sub_collection\archive.txt"
f_series_path = r"data\sub_collection\f_series.txt"
federal_non_nih_path = r"data\sub_collection\federal_non_nih.txt"
grant_resources_path = r"data\sub_collection\grant_resources.txt"
industry_path = r"data\sub_collection\industry.txt"
k_series_path = r"data\sub_collection\k_series.txt"
r_series_path = r"data\sub_collection\r_series.txt"
sample_path = r"data\sub_collection\sample_.txt"
t_series_path = r"data\sub_collection\t_series.txt"
u_series_path = r"data\sub_collection\u_series.txt"

problem_sub_dict_list = []

path_sub_list = [archive_path, f_series_path, federal_non_nih_path, grant_resources_path,industry_path, k_series_path,
            r_series_path, sample_path, t_series_path,u_series_path]

for path in path_sub_list: 
#     print(path)
    with open(path, 'r', encoding= 'utf8') as f:
        s = f.read()
#         problem_sub_dict = json.loads(data)
#         problem_sub_dict_list.append(problem_sub_dict)

        while True:
            try:
                result = json.loads(s)   # try to parse...
                break                    # parsing worked -> exit loop
            except Exception as e:
                # "Expecting , delimiter: line 34 column 54 (char 1158)"
                # position of unexpected character after '"'
                unexp = int(re.findall(r'\(char (\d+)\)', str(e))[0])
                # position of unescaped '"' before that
                unesc = s.rfind(r'"', 0, unexp)
                s = s[:unesc] + r'\"' + s[unesc+1:]
                # position of correspondig closing '"' (+2 for inserted '\')
                closg = s.find(r'"', unesc + 2)
                s = s[:closg] + r'\"' + s[closg+1:]
#         print(result)
        problem_sub_dict_list.append(result)



## https://stackoverflow.com/questions/16573332/jsondecodeerror-expecting-value-line-1-column-1-char-0
## https://stackoverflow.com/questions/18514910/how-do-i-automatically-fix-an-invalid-json-string

## Checked: No problems

In [30]:
problem_sub_df = pd.DataFrame.from_dict(json_normalize(problem_sub_dict_list, max_level=1))
problem_sub_df.head()

## Checked: No problems

C:\Users\keg827\AppData\Local\Temp\ipykernel_30316\2473597912.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  problem_sub_df = pd.DataFrame.from_dict(json_normalize(problem_sub_dict_list, max_level=1))


,Multi-page?,Title,Keyword,Resource type(s),Rights,Creator,Contributor,Description,Abstract,Original Bibliographic Citation,...,Language,Subject: MESH,Subject: LCSH,Subject: Geographic Name,Subject: Name,Location,Digital Origin,Id,uri,members
0,False,Archive,[archive],[],[All rights reserved],"[Traw, Emily, Herzog, Keith Alan, Gutzman, Kar...","[Traw, Emily, Gutzman, Karen E]",,[],[],...,[],[],[],[],[],[],[],46374bec-fe29-4a06-a8b8-f1f93668699c,https://digitalhub.northwestern.edu/collection...,[]
1,False,F-Series,"[F Award, F-Series]",[],[All rights reserved],[],[],,[],[],...,[],[],[],[],[],[],[],1b2b7d0f-d7f3-41a4-a2a4-8435d719ecec,https://digitalhub.northwestern.edu/collection...,[{'Title': ['DeLosAngeles_Wang_summary stateme...
2,None,Federal Non-NIH Awards,"[peer-reviewed grants, research support, grant...",[],[],[],[],None,[],[],...,[],[],[],[],[],[],[],9aa9dbb6-7971-4e5c-9b9c-81216d9325d3,https://digitalhub.northwestern.edu/collection...,"[{'Title': ['Smith_U18_5.26.2016'], 'Resource ..."
3,False,Grant Resources,"[Biosketch, Resources and Environment, RCR]",[],[All rights reserved],"[Traw, Emily]","[Traw, Emily]",,[],[],...,[],[],[],[],[],[],[],691ddbbd-785c-4a6a-85d1-7569d8fcbcdb,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Akono_NIH Diversity Supplement_4....
4,False,Industry/Foundation/Other,"[research support, grant proposals, Industry, ...",[],[],[],[],,[],[],...,[],[],[],[],[],[],[],01b2ba97-8603-477f-bde3-b36b6c79bdea,https://digitalhub.northwestern.edu/collection...,"[{'Multi-page?': None, 'Title': ['child-0-of-t..."


In [31]:
## Concatenate the DigitalHub Community Dataframe to the problem_df

digitalhub_sub_collection_recall_df = pd.concat([digitalhub_sub_collection_recall_df, problem_sub_df], axis=0)
digitalhub_sub_collection_recall_df.reset_index(inplace=True, drop=True) 
digitalhub_sub_collection_recall_df.head(10)

## Checked: No problems

,Multi-page?,Title,Keyword,Resource type(s),Rights,Creator,Contributor,Description,Abstract,Original Bibliographic Citation,...,Language,Subject: MESH,Subject: LCSH,Subject: Geographic Name,Subject: Name,Location,Digital Origin,Id,uri,members
0,False,Fall 2017,"[medical research, Biostatistics Collaboration...",[],[All rights reserved],[],[],Free introductory lectures on biostatistics in...,[],[],...,[],[],[],[],[],[],[],17c76dde-4be6-4d2d-9020-3904a9451617,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Biostatistics 101: the minor deta...
1,False,Winter 2019,"[Biostatistics Collaboration Center , medical ...",[],[All rights reserved],[],[],Free introductory lectures on biostatistics in...,[],[],...,[],[],[],[],[],[],[],5d8b5f8e-c46b-4e97-a5a2-9c118c9cca24,https://digitalhub.northwestern.edu/collection...,[{'Title': ['A Statistician's Guide to REDCap'...
2,False,Fall 2016,"[Biostatistics Collaboration Center , biostati...",[],[All rights reserved],[],[],Free introductory lectures on biostatistics in...,[],[],...,[],[],[],[],[],[],[],fc389d13-2430-409b-82fd-a4b26613d350,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Clinical Trials: Highlights from ...
3,True,2019-2020,"[Biostatistics Collaboration Center, Statistic...",[],[All rights reserved],[],[],Free introductory lectures on biostatistics in...,[],[],...,[],[Biostatistics],[],[],[],[],[],a86e1412-d72c-4cae-b8ca-16fd834cb128,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Logistic Regression: Odds & Ends'...
4,None,Chicago Biomedical Informatics and Data Scienc...,"[NUCATS, CDSI, informatics, data science]",[],[],[],[],Biomedical and Health Informatics and Data Sci...,[],[],...,[],[],[],[],[],[],[],6d66813c-a6e8-49b0-a568-d9378f82da51,https://digitalhub.northwestern.edu/collection...,[]
5,False,Biomedical Data Science Day,"[informatics, CDSI, data science, NUCATS, biom...",[],[],"[Soulakis, Nicholas Dean]",[],The Center for Data Science and Informatics pr...,[],[],...,[],[],[],[],[],[],[],ab570598-e497-4c81-9351-1aa316252682,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Breaking Data Silos: the Gen3 Pla...
6,True,Northwestern University Medical School 1859-1979,[History],[],[All rights reserved],"[Arey, Leslie Brainerd, 1891-]",[],"A history of the Feinberg School of Medicine, ...",[],[],...,[English],"[History, Schools, Medical]",[],[Chicago (Ill.)],"[Northwestern University (Evanston, Ill.). Med...",[],[],areybook,https://digitalhub.northwestern.edu/collection...,"[{'Title': ['Preface'], 'Resource type(s)': ['..."
7,False,Library Notes,"[Galter Health Sciences Library, Newsletter, A...",[],[All rights reserved],[Galter Health Sciences Library],[],Newsletter published by Galter Health Sciences...,[],[],...,[English],"[Chicago, Libraries, Medical]",[],[],[],[],[],library-notes,https://digitalhub.northwestern.edu/collection...,"[{'Title': ['Library Notes, April/May/June 199..."
8,None,"Dissemination, Impact, and Access Workshop 2017",[workshop],[],[],[],[],None,[],[],...,[],[],[],[],[],[],[],diaw2017,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Complying with the NIH Public Acc...
9,False,12th General Hospital Collection,[World War II],[],[],"[Connor, James A., Mason, Michael L., 1895-1963]",[],The 12th General Hospital Collection is compri...,[],[],...,[English],"[Hospitals, Military]","[World War, 1939-1945]","[Naples (Italy), Rome (Italy), Livorno (Italy)...",[12th General Hospital],[],[],07b25bee-4a47-466a-b9b8-70d7a392fab0,https://digitalhub.northwestern.edu/collection...,"[{'Multi-page?': False, 'Title': ['James A. Co..."


In [32]:
## Add Column to digitalhub_sub_collection_df to indicate that these results are a sub_collection
digitalhub_sub_collection_recall_df["Level Type"] = "Sub_Collection"
digitalhub_sub_collection_recall_df["Level Number"] = "2"
digitalhub_sub_collection_recall_df['Level Number'] = digitalhub_sub_collection_recall_df['Level Number'].apply(int)

## Create a column from the index
digitalhub_sub_collection_recall_df['sub_collection_rowid'] = digitalhub_sub_collection_recall_df.index

## Create a new column called Sub_Collection_ID
digitalhub_sub_collection_recall_df['Sub_Collection_ID'] = digitalhub_sub_collection_recall_df['Id']

## Checked: No problems

In [33]:
## Inspect the "members" column and create a list of member IDs

row_sub_collection_member_list = []
column_sub_collection_member_dict ={}
count_sub_collection_member_dict = {}


for k, v in digitalhub_sub_collection_recall_df["members"].items():
    for value in v: 
        member = value["Id"]
        row_sub_collection_member_list.append(member)
        count_sub_collection_member_dict[k] = len(row_sub_collection_member_list)
    column_sub_collection_member_dict[k] = row_sub_collection_member_list
    row_sub_collection_member_list =[]
   
## Append the column member dictionary to the digitalhub_sub_collection_recall_df dataframe

digitalhub_sub_collection_recall_df['Member_List'] = digitalhub_sub_collection_recall_df.index.map(column_sub_collection_member_dict)
digitalhub_sub_collection_recall_df['Member_List_Count'] = digitalhub_sub_collection_recall_df.index.map(count_sub_collection_member_dict)
# digitalhub_sub_collection_recall_df.head()

## Checked: No problems



In [34]:
## Export file to excel, without the Pandas index, but with the header

digitalhub_sub_collection_recall_df.to_excel("outputs/digitalhub_sub_collection_recall_df.xlsx", index=False, header=True)

##### Split Recall Sub_Collection into Members

In [35]:
## Create a dataframe for each members row and concatenate all of these into one member_df

sub_collection_members_dfs_list = []

for k, v in digitalhub_sub_collection_recall_df["members"].items():
#     print("this is k: ", k)
#     print("this is v: ", v)
    sub_collection_member_df = pd.json_normalize(v)
    sub_collection_member_df['sub_collection_rowid'] = k
    sub_collection_members_dfs_list.append(sub_collection_member_df)

digitalhub_sub_collection_members_df = pd.concat(sub_collection_members_dfs_list, sort=False).reset_index(drop='index')
# digitalhub_sub_collection_members_df.head()

## Resource
## https://stackoverflow.com/questions/62816027/convert-pandas-json-column-to-multiple-rows

## Checked: No problems

In [36]:
## Add Column to digitalhub_community_df to indicate that these results are a collection

digitalhub_sub_collection_members_df['Level Type'] = np.where(digitalhub_sub_collection_members_df['DOI'].isnull(),"Sub_Sub_Collection", "Record" )
digitalhub_sub_collection_members_df['Level Number'] = np.where(digitalhub_sub_collection_members_df['DOI'].isnull(),"3", "6" )
digitalhub_sub_collection_members_df['Level Number'] = digitalhub_sub_collection_members_df['Level Number'].apply(int)
# digitalhub_sub_collection_members_df.head()

## Checked: No problems

In [37]:
## Export file to excel with the header

digitalhub_sub_collection_members_df.to_excel("outputs/digitalhub_sub_collection_members_df.xlsx", header=True)

##### Concatenate Recall Sub_Collection with Members

In [38]:
## Concatenate the DigitalHub Sub_Collection_Recall Dataframe using the 'rowid' to the DigitalHub Sub_Collection Members Dataframe using the 'rowid'

digitalhub_sub_col_members_df = pd.concat([digitalhub_sub_collection_recall_df, digitalhub_sub_collection_members_df], axis=0).sort_values(by=['sub_collection_rowid'])
# digitalhub_sub_col_members_df.head()

## Checked: No problems

In [39]:
## Sort the dataframe by sub_collection_rowid and Level Number to prepare for group by and forward fill of Sub_Collection_ID

digitalhub_sub_col_members_df.sort_values(by = ['sub_collection_rowid','Level Number'], ascending = [True, True], inplace=True)

## Checked: No problems

In [40]:
## Groupby Sub_Collection_rowid and fill forward the Sub_Collection_ID into members of sub_collections (i.e. sub_sub_collections and records)

digitalhub_sub_col_members_df.update(digitalhub_sub_col_members_df.groupby(['sub_collection_rowid'])['Sub_Collection_ID'].ffill())

## Resources
## https://stackoverflow.com/questions/64795941/how-do-i-forward-fill-nas-with-condition-of-2-other-cells-being-equal-in-pandas
## https://stackoverflow.com/questions/27012151/forward-fill-specific-columns-in-pandas-dataframe

## Checked: No problems

In [41]:
## Create a column for Sub_Sub_Collection_ID to hold the Id from columns with Level Type as "Sub_Sub_Collection"

digitalhub_sub_col_members_df['Sub_Sub_Collection_ID'] = np.where(digitalhub_sub_col_members_df['Level Type'] == 'Sub_Sub_Collection',digitalhub_sub_col_members_df['Id'], np.nan)

## Resources
## https://stackoverflow.com/questions/67043249/how-to-use-np-where-in-creating-new-column-using-previous-rows

## Checked: No problems

In [42]:
## Export file to excel, without the Pandas index, but with the header

digitalhub_sub_col_members_df.to_excel("outputs/digitalhub_sub_col_members_df.xlsx", index=False, header=True)

## Checked: No problems

##### Concatenate Sub_Collection with Community

In [43]:
## Concatenate the DigitalHub digitalhub_comm_col_df Dataframe using the 'Id' to the DigitalHub digitalhub_sub_col_members_df Dataframe using the 'Id'

#digitalhub_comm_col_df
#digitalhub_sub_col_members_df

digitalhub_sub_col_sub_col_df = pd.concat([digitalhub_comm_col_df, digitalhub_sub_col_members_df], axis=0).sort_values(by=['Community_ID', 'Sub_Collection_ID'])
# digitalhub_sub_col_sub_col_df.head()

## Checked: No problems
## Note: Creates duplicate rows for "Sub_Collection_ID"


In [44]:
## Sort the dataframe by Id and Level Number to get ready to groupby ID and remove duplicate Sub_Collections from the concatenate

digitalhub_sub_col_sub_col_df.sort_values(by = ['Id','Level Number'], ascending = [True, True], inplace=True)

## Checked: No problems

In [45]:
# Remove the duplicate Sub_Collections based on their Id and take the first non-null data

digitalhub_sub_col_sub_col_df = digitalhub_sub_col_sub_col_df.groupby(['Id'], as_index=False).first().reset_index()

## Resources: 
## https://stackoverflow.com/questions/64795941/how-do-i-forward-fill-nas-with-condition-of-2-other-cells-being-equal-in-pandas
## https://www.pauldesalvo.com/how-to-apply-a-forward-fill-ffill-to-groups-in-pandas/

## Checked: No problems

In [46]:
## Sort the dataframe by Sub_Collection_ID and then Level Number to get ready to fill Community IDs into connected Sub_Collections

digitalhub_sub_col_sub_col_df.sort_values(by = ['Sub_Collection_ID','Level Number'], ascending = [True, True], inplace=True)

## Checked: No problems

In [47]:
## Fill foward the Community_ID into Records based on their Sub_Collection_IDs

digitalhub_sub_col_sub_col_df.update(digitalhub_sub_col_sub_col_df.groupby(['Sub_Collection_ID'])['Community_ID'].ffill())

## Other options
##digitalhub_sub_col_sub_col_df['Community_ID'] = digitalhub_sub_col_sub_col_df.groupby(['Sub_Collection_ID'])['Community_ID'].fillna(method='ffill')
##digitalhub_sub_col_sub_col_df['Community_ID'] = digitalhub_sub_col_sub_col_df.groupby(['Sub_Collection_ID'])['Community_ID'].transform(lambda x: x.ffill())
##digitalhub_sub_col_sub_col_df['Community_ID'] = digitalhub_sub_col_sub_col_df.groupby(['Sub_Collection_ID'])['Community_ID'].fillna(method='ffill')

## Resources
## https://stackoverflow.com/questions/64795941/how-do-i-forward-fill-nas-with-condition-of-2-other-cells-being-equal-in-pandas
## https://stackoverflow.com/questions/58181262/groupby-with-ffill-deletes-group-and-does-not-put-group-in-index

## Checked: No problems

In [48]:
## Export file to excel with the header

digitalhub_sub_col_sub_col_df.to_excel("outputs/digitalhub_sub_col_sub_col_df.xlsx",  header=True)

## Checked: No problems

In [49]:
## DO NOT NEED
## Because of groupby issue from above, re-add the community id for communities that lost theirs

## Sort the dataframe by rowid and Level Number
# digitalhub_sub_col_sub_col_df.sort_values(by = ['community_rowid','Level Number'], ascending = [True, True], inplace=True)

# ## Create a column for Sub_Collection ID
# digitalhub_sub_col_sub_col_df['Community_ID'] = np.where(digitalhub_sub_col_sub_col_df['Level Type'] == 'Community',digitalhub_sub_col_sub_col_df['Id'], np.nan)

# ## https://stackoverflow.com/questions/67043249/how-to-use-np-where-in-creating-new-column-using-previous-rows

# # ## Fill forward the Community ID into subcollections and records
# digitalhub_sub_col_sub_col_df['Community_ID'] = digitalhub_sub_col_sub_col_df.groupby(['community_rowid'])['Community_ID'].ffill()

# ## https://stackoverflow.com/questions/64795941/how-do-i-forward-fill-nas-with-condition-of-2-other-cells-being-equal-in-pandas


In [50]:
## DO NOT NEED
## Export file to excel, without the Pandas index, but with the header

# digitalhub_sub_col_sub_col_df.to_excel("outputs/digitalhub_sub_col_sub_col_df.xlsx",  header=True)

##### Recall Sub_Sub_Collection

In [51]:
## Extract a the Series for sub_sub_collection "dh_id" and transform into a list

digitalhub_sub_sub_collection_series = digitalhub_sub_col_sub_col_df[digitalhub_sub_col_sub_col_df["Level Type"] == "Sub_Sub_Collection"]['Id']
digitalhub_sub_sub_collection_list = digitalhub_sub_sub_collection_series.tolist()

print(digitalhub_sub_sub_collection_list)

## Checked: No problems

['8b34974a-1cef-402d-8a57-c04c1f015fa9', 'af7512ff-addc-457c-b2c2-22f49b3aaab0', 'eaf6729a-34d6-4dec-ae66-c339f14fe1ea', '906b9f6e-a70e-4821-aee0-9a038ec909d5', 'a4ba4155-9cd8-43eb-b4a7-865e50b61dc0', 'c2b21560-15fb-4c7d-a7ca-2ecee215a647', '49945521-3b35-4efd-9e77-e909900f1604', '6a9690f5-664a-4654-a5ca-c1c1ca99db9b', 'af85d02b-4b6c-48db-8cac-b82f4075e1f0', '7d40546b-cad0-4838-b289-947b5187c02a', 'cb52ca3c-f967-41b1-886d-60f5611706ac', 'c6f1d0a2-9bc2-424f-aebe-1ea6beed176d', '102d9812-55fa-4aeb-b910-04e534f7a7ef', '7b81356b-6eaa-4570-bd18-355714700aed', '898c31c9-b407-46e9-b923-7bd7f7fdae42', 'aed04ba0-5b58-4a10-94de-cc062d44bcfc', 'b37f04ab-f9d6-4e97-a896-6e63d0586c9b', '7a934737-c0e7-4ed3-83d2-d81ed70a72fe', 'areybook0', 'areybook1', 'areybook10', 'areybook11', 'areybook12', 'areybook13', 'areybook14', 'areybook15', 'areybook16', 'areybook17', 'areybook18', 'areybook19', 'areybook2', 'areybook20', 'areybook21', 'areybook3', 'areybook4', 'areybook5', 'areybook6', 'areybook7', 'areybo

In [52]:
## Loop through list of DigitalHub Sub_Sub_Collections and use the urllib.request to get the json data
## test_list = ['2cc92425-b656-47ea-a3b4-825405ee6088', 'a86e1412-d72c-4cae-b8ca-16fd834cb128','fc389d13-2430-409b-82fd-a4b26613d350']

multi_digitalhub_sub_sub_collection_list = []
digitalhub_sub_sub_collection_problem_list = []

for item in digitalhub_sub_sub_collection_list:
    try:

        with urllib.request.urlopen(f"https://digitalhub.northwestern.edu/collections/{item}.json" ) as url:
            single_digitalhub_sub_sub_collection_dict = json.loads(url.read().decode())
            multi_digitalhub_sub_sub_collection_list.append(single_digitalhub_sub_sub_collection_dict)
            print(item)
        time.sleep(1)
       

    except urllib.error.HTTPError as http_err:
        print(item)
        digitalhub_sub_sub_collection_problem_list.append(item)
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
        

    except urllib.error.URLError as url_err:
        print(item)
        digitalhub_sub_sub_collection_problem_list.append(item)
        print(f'URL error occurred: {url_err}. ', 'Exiting the loop!')  # Python 3.6
       

    except json.JSONDecodeError as json_err:
        print(item)
        digitalhub_sub_sub_collection_problem_list.append(item)
        print(f'JSON Decode error occurred: {json_err}. ', 'Poorly formed JSON.')  # Python 3.6
       
        
    except Exception as err:
        print(item)
        digitalhub_sub_sub_collection_problem_list.append(item)
        print(f'Other error occurred: {err}. ')  # Python 3.6
        
        
       
    else:
        print('Success!')



## Resources
## https://docs.python.org/3/library/urllib.request.html

## Checked: No problems

8b34974a-1cef-402d-8a57-c04c1f015fa9
JSON Decode error occurred: Expecting value: line 1 column 2 (char 1).  Poorly formed JSON.
af7512ff-addc-457c-b2c2-22f49b3aaab0
JSON Decode error occurred: Expecting value: line 1 column 2 (char 1).  Poorly formed JSON.
eaf6729a-34d6-4dec-ae66-c339f14fe1ea
JSON Decode error occurred: Expecting value: line 1 column 2 (char 1).  Poorly formed JSON.
906b9f6e-a70e-4821-aee0-9a038ec909d5
JSON Decode error occurred: Expecting value: line 1 column 2 (char 1).  Poorly formed JSON.
a4ba4155-9cd8-43eb-b4a7-865e50b61dc0
JSON Decode error occurred: Expecting value: line 1 column 2 (char 1).  Poorly formed JSON.
c2b21560-15fb-4c7d-a7ca-2ecee215a647
JSON Decode error occurred: Expecting value: line 1 column 2 (char 1).  Poorly formed JSON.
49945521-3b35-4efd-9e77-e909900f1604
JSON Decode error occurred: Expecting value: line 1 column 2 (char 1).  Poorly formed JSON.
6a9690f5-664a-4654-a5ca-c1c1ca99db9b
JSON Decode error occurred: Expecting value: line 1 column 2

pnb-17-5
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-17-6
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-17-7
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-17-8
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-17-9
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-18-1
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-18-10
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-18-11
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-18-12
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-18-2
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-18-3
URL erro

pnb-24-3
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-24-4
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-24-5
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-24-6
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-24-7
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-24-8
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-24-9
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-25-1
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-25-10
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-25-11
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-25-12
URL erro

pnb-4-6
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-4-7
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-4-8
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-4-9
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-5-1
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-5-10
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-5-11
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-5-12
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-5-2
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-5-3
URL error occurred: <urlopen error [Errno 11001] getaddrinfo failed>.  Exiting the loop!
pnb-5-4
URL error occurred:

In [53]:
## Inspect the results of the URL Query for DigitalHub Collections that will become communities in Prism

# print(multi_digitalhub_sub_sub_collection_list)
print(digitalhub_sub_sub_collection_problem_list)

['8b34974a-1cef-402d-8a57-c04c1f015fa9', 'af7512ff-addc-457c-b2c2-22f49b3aaab0', 'eaf6729a-34d6-4dec-ae66-c339f14fe1ea', '906b9f6e-a70e-4821-aee0-9a038ec909d5', 'a4ba4155-9cd8-43eb-b4a7-865e50b61dc0', 'c2b21560-15fb-4c7d-a7ca-2ecee215a647', '49945521-3b35-4efd-9e77-e909900f1604', '6a9690f5-664a-4654-a5ca-c1c1ca99db9b', 'af85d02b-4b6c-48db-8cac-b82f4075e1f0', '7d40546b-cad0-4838-b289-947b5187c02a', 'cb52ca3c-f967-41b1-886d-60f5611706ac', 'c6f1d0a2-9bc2-424f-aebe-1ea6beed176d', 'areybook0', 'areybook1', 'areybook10', 'areybook11', 'areybook12', 'areybook13', 'areybook14', 'areybook15', 'areybook16', 'areybook17', 'areybook18', 'areybook19', 'areybook2', 'areybook20', 'areybook21', 'areybook3', 'areybook4', 'areybook5', 'areybook6', 'areybook7', 'areybook8', 'areybook9', 'pnb-14-2', 'pnb-14-3', 'pnb-14-4', 'pnb-14-5', 'pnb-14-6', 'pnb-14-7', 'pnb-14-8', 'pnb-14-9', 'pnb-15-1', 'pnb-15-10', 'pnb-15-11', 'pnb-15-12', 'pnb-15-2', 'pnb-15-3', 'pnb-15-4', 'pnb-15-5', 'pnb-15-6', 'pnb-15-7', 'p

In [54]:
## Create a dataframe from DigitalHub json for DigitalHub Sub_Sub_Collections

digitalhub_sub_sub_collection_recall_df = pd.DataFrame.from_dict(json_normalize(multi_digitalhub_sub_sub_collection_list, max_level=1))
digitalhub_sub_sub_collection_recall_df.head(10)

C:\Users\keg827\AppData\Local\Temp\ipykernel_30316\3779115508.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  digitalhub_sub_sub_collection_recall_df = pd.DataFrame.from_dict(json_normalize(multi_digitalhub_sub_sub_collection_list, max_level=1))


,Multi-page?,Title,Keyword,Resource type(s),Rights,Creator,Contributor,Description,Abstract,Original Bibliographic Citation,...,Language,Subject: MESH,Subject: LCSH,Subject: Geographic Name,Subject: Name,Location,Digital Origin,Id,uri,members
0,False,Conner Photographs,"[12th General Hospital, World War II]",[],[],[Creator not identified],[],Photographs of James A. Conner.,[],[],...,[],[],[],[],"[Conner, James A. (James Augustus), 1903-2001]",[],[],102d9812-55fa-4aeb-b910-04e534f7a7ef,https://digitalhub.northwestern.edu/collection...,"[{'Title': [' Portrait of James A. Conner'], '..."
1,False,Conner Insignia,"[12th General Hospital, World War II]",[],[http://creativecommons.org/publicdomain/mark/...,[United States. Army],[],This Sub-collection contains insignia and badg...,[],[],...,[],[],[United States. Army--Insignia],[],[],[],[],7b81356b-6eaa-4570-bd18-355714700aed,https://digitalhub.northwestern.edu/collection...,[{'Title': ['WWII U.S. Army Eagle Officer Hat ...
2,False,Conner Selected Documents,"[World War II, 12th General Hospital, Pediatri...",[],[],[],[],"This sub-collection includes receipts, poetry,...",[],[],...,[English],[],[],[],[],[],[],898c31c9-b407-46e9-b923-7bd7f7fdae42,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Appointment Letter to Be in Charg...
3,False,12th General Hospital Medical Officer Biographies,[Medical Officers],[],[http://creativecommons.org/publicdomain/mark/...,[],[],This sub-collection includes photographs and r...,[],[],...,[English],"[Physicians, Surgeons, Nurses, Physical Therap...","[World War, 1939-1945]",[],[12th General Hospital],[],[],aed04ba0-5b58-4a10-94de-cc062d44bcfc,https://digitalhub.northwestern.edu/collection...,"[{'Title': ['Anne Ponovich'], 'Resource type(s..."
4,False,Conner Ephemera,"[12th General Hospital, World War II]",[],[All rights reserved],[],[],This sub-collection contains ephemera and souv...,[],[],...,[],[],"[Souvenirs (Keepsakes), World War, 1939-1945]",[],[ ],[],[],b37f04ab-f9d6-4e97-a896-6e63d0586c9b,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Army Exchange Service Drink Card'...
5,False,Feinberg Biomedical Data Science Day 2020,"[data science, CDSI, informatics, NUCATS, biom...",[],[],[],[],The Center for Data Science and Informatics pr...,[],[],...,[],[],[],[],[],[],[],7a934737-c0e7-4ed3-83d2-d81ed70a72fe,https://digitalhub.northwestern.edu/collection...,[]
6,False,Mason Postcards,"[12th General Hospital, World War II, Sitseein...",[],[],[],[],This sub-collection includes postcards from It...,[],[],...,"[French, German, Italian]",[],[],[],[],[],[],04227302-08c6-475a-8ec9-3e2f18907c24,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Breton et Bretonne fumant la pipe...
7,False,MEDI-CALL,[Newsletter],[],[http://creativecommons.org/publicdomain/mark/...,[12th General Hospital],[],Weekly newsletter of the 12th General Hospital...,[],[],...,[English],"[Military Medicine, Hospitals, Military]","[World War, 1939-1945--Literature and the war]",[],[12th General Hospital],"[Livorno, Tuscany, Italy]",[],129331fe-295c-422b-908c-822700382bac,https://digitalhub.northwestern.edu/collection...,"[{'Title': ['MEDI-CALL. Vol. 1, No. 10 (18 Mar..."
8,False,Mason Photographs,"[surgery, 12th General Hospital, World War II,...",[],[],[],[],,[],[],...,[],"[Military Medicine--history, World War II, Hos...",[],[],[12th General Hospital],[],[],19e0660b-7dd3-480b-b3ec-b57a12f34014,https://digitalhub.northwestern.edu/collection...,"[{'Title': ['Ain-El-Turck 001'], 'Resource typ..."
9,False,12th General Hospital reports,"[12th General Hospital, World War II, Statistics]",[],[http://creativecommons.org/publicdomain/mark/...,[],[],This collection includes monthly accounts of ...,[],[],...,[English],"[Surgical Procedures, Operative, Epidemiology,...",[],[],[],[],[],1a3c738a-ba69-4275-98b0-fc77939d5b93,https://digitalhub.northwestern.edu/collection...,[{'Title': ['12th General Hospital final repor...


In [55]:
## Upload .txt files of the problem Sub_Sub_Collection json metadata

biosketches_path = r"data\sub_sub_collection\biosketches.txt"
R01_path = r"data\sub_sub_collection\R01.txt"
R03_path = r"data\sub_sub_collection\R03.txt"
R21_path = r"data\sub_sub_collection\R21.txt"
summary_statements_path = r"data\sub_sub_collection\summary_statements.txt"


problem_sub_sub_dict_list = []

path_sub_sub_list = [biosketches_path, R01_path, R03_path, R21_path, summary_statements_path]

for path in path_sub_sub_list: 
#     print(path)
    with open(path, 'r', encoding= 'utf8') as f:
        s = f.read()
#         problem_sub_sub_dict = json.loads(data)
#         problem_sub_sub_dict_list.append(problem_sub_dict)

        while True:
            try:
                result = json.loads(s)   # try to parse...
                break                    # parsing worked -> exit loop
            except Exception as e:
                # "Expecting , delimiter: line 34 column 54 (char 1158)"
                # position of unexpected character after '"'
                unexp = int(re.findall(r'\(char (\d+)\)', str(e))[0])
                # position of unescaped '"' before that
                unesc = s.rfind(r'"', 0, unexp)
                s = s[:unesc] + r'\"' + s[unesc+1:]
                # position of correspondig closing '"' (+2 for inserted '\')
                closg = s.find(r'"', unesc + 2)
                s = s[:closg] + r'\"' + s[closg+1:]
#         print(result)
        problem_sub_sub_dict_list.append(result)



## https://stackoverflow.com/questions/16573332/jsondecodeerror-expecting-value-line-1-column-1-char-0
## https://stackoverflow.com/questions/18514910/how-do-i-automatically-fix-an-invalid-json-string

## Checked: No problems

In [56]:
problem_sub_sub_df = pd.DataFrame.from_dict(json_normalize(problem_sub_sub_dict_list, max_level=1))
problem_sub_sub_df.head()

## Checked: No problems

C:\Users\keg827\AppData\Local\Temp\ipykernel_30316\3852235292.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  problem_sub_sub_df = pd.DataFrame.from_dict(json_normalize(problem_sub_sub_dict_list, max_level=1))


,Multi-page?,Title,Keyword,Resource type(s),Rights,Creator,Contributor,Description,Abstract,Original Bibliographic Citation,...,Language,Subject: MESH,Subject: LCSH,Subject: Geographic Name,Subject: Name,Location,Digital Origin,Id,uri,members
0,None,Biosketches,"[peer-reviewed grants, research support, grant...",[],[],[],[],None,[],[],...,[],[],[],[],[],[],[],7d40546b-cad0-4838-b289-947b5187c02a,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Cianciotto R21 biosketch 3.14.201...
1,False,R01,"[peer-reviewed grants, research support, grant...",[],[],[],[],,[],[],...,[],[],[],[],[],[],[],6a9690f5-664a-4654-a5ca-c1c1ca99db9b,https://digitalhub.northwestern.edu/collection...,[]
2,None,R03,"[peer-reviewed grants, research support, grant...",[],[],[],[],None,[],[],...,[],[],[],[],[],[],[],49945521-3b35-4efd-9e77-e909900f1604,https://digitalhub.northwestern.edu/collection...,[]
3,None,R21,"[peer-reviewed grants, research support, grant...",[],[],[],[],None,[],[],...,[],[],[],[],[],[],[],af85d02b-4b6c-48db-8cac-b82f4075e1f0,https://digitalhub.northwestern.edu/collection...,[]
4,False,Summary Statements,"[peer-reviewed grants, research support, grant...",[],[],[],[],,[],[],...,[],[],[],[],[],[],[],cb52ca3c-f967-41b1-886d-60f5611706ac,https://digitalhub.northwestern.edu/collection...,[{'Title': ['DeLosAngeles_Wang_summary stateme...


In [57]:
## Concatenate the DigitalHub Community Dataframe to the problem_df

digitalhub_sub_sub_collection_recall_df= pd.concat([digitalhub_sub_sub_collection_recall_df, problem_sub_sub_df], axis=0)
digitalhub_sub_sub_collection_recall_df.reset_index(inplace=True, drop=True) 
digitalhub_sub_sub_collection_recall_df.head(10)

## Checked: No problems

,Multi-page?,Title,Keyword,Resource type(s),Rights,Creator,Contributor,Description,Abstract,Original Bibliographic Citation,...,Language,Subject: MESH,Subject: LCSH,Subject: Geographic Name,Subject: Name,Location,Digital Origin,Id,uri,members
0,False,Conner Photographs,"[12th General Hospital, World War II]",[],[],[Creator not identified],[],Photographs of James A. Conner.,[],[],...,[],[],[],[],"[Conner, James A. (James Augustus), 1903-2001]",[],[],102d9812-55fa-4aeb-b910-04e534f7a7ef,https://digitalhub.northwestern.edu/collection...,"[{'Title': [' Portrait of James A. Conner'], '..."
1,False,Conner Insignia,"[12th General Hospital, World War II]",[],[http://creativecommons.org/publicdomain/mark/...,[United States. Army],[],This Sub-collection contains insignia and badg...,[],[],...,[],[],[United States. Army--Insignia],[],[],[],[],7b81356b-6eaa-4570-bd18-355714700aed,https://digitalhub.northwestern.edu/collection...,[{'Title': ['WWII U.S. Army Eagle Officer Hat ...
2,False,Conner Selected Documents,"[World War II, 12th General Hospital, Pediatri...",[],[],[],[],"This sub-collection includes receipts, poetry,...",[],[],...,[English],[],[],[],[],[],[],898c31c9-b407-46e9-b923-7bd7f7fdae42,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Appointment Letter to Be in Charg...
3,False,12th General Hospital Medical Officer Biographies,[Medical Officers],[],[http://creativecommons.org/publicdomain/mark/...,[],[],This sub-collection includes photographs and r...,[],[],...,[English],"[Physicians, Surgeons, Nurses, Physical Therap...","[World War, 1939-1945]",[],[12th General Hospital],[],[],aed04ba0-5b58-4a10-94de-cc062d44bcfc,https://digitalhub.northwestern.edu/collection...,"[{'Title': ['Anne Ponovich'], 'Resource type(s..."
4,False,Conner Ephemera,"[12th General Hospital, World War II]",[],[All rights reserved],[],[],This sub-collection contains ephemera and souv...,[],[],...,[],[],"[Souvenirs (Keepsakes), World War, 1939-1945]",[],[ ],[],[],b37f04ab-f9d6-4e97-a896-6e63d0586c9b,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Army Exchange Service Drink Card'...
5,False,Feinberg Biomedical Data Science Day 2020,"[data science, CDSI, informatics, NUCATS, biom...",[],[],[],[],The Center for Data Science and Informatics pr...,[],[],...,[],[],[],[],[],[],[],7a934737-c0e7-4ed3-83d2-d81ed70a72fe,https://digitalhub.northwestern.edu/collection...,[]
6,False,Mason Postcards,"[12th General Hospital, World War II, Sitseein...",[],[],[],[],This sub-collection includes postcards from It...,[],[],...,"[French, German, Italian]",[],[],[],[],[],[],04227302-08c6-475a-8ec9-3e2f18907c24,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Breton et Bretonne fumant la pipe...
7,False,MEDI-CALL,[Newsletter],[],[http://creativecommons.org/publicdomain/mark/...,[12th General Hospital],[],Weekly newsletter of the 12th General Hospital...,[],[],...,[English],"[Military Medicine, Hospitals, Military]","[World War, 1939-1945--Literature and the war]",[],[12th General Hospital],"[Livorno, Tuscany, Italy]",[],129331fe-295c-422b-908c-822700382bac,https://digitalhub.northwestern.edu/collection...,"[{'Title': ['MEDI-CALL. Vol. 1, No. 10 (18 Mar..."
8,False,Mason Photographs,"[surgery, 12th General Hospital, World War II,...",[],[],[],[],,[],[],...,[],"[Military Medicine--history, World War II, Hos...",[],[],[12th General Hospital],[],[],19e0660b-7dd3-480b-b3ec-b57a12f34014,https://digitalhub.northwestern.edu/collection...,"[{'Title': ['Ain-El-Turck 001'], 'Resource typ..."
9,False,12th General Hospital reports,"[12th General Hospital, World War II, Statistics]",[],[http://creativecommons.org/publicdomain/mark/...,[],[],This collection includes monthly accounts of ...,[],[],...,[English],"[Surgical Procedures, Operative, Epidemiology,...",[],[],[],[],[],1a3c738a-ba69-4275-98b0-fc77939d5b93,https://digitalhub.northwestern.edu/collection...,[{'Title': ['12th General Hospital final repor...


In [58]:
## Add Column to digitalhub_sub_sub_collection_df to indicate that these results are a sub_sub_collection
digitalhub_sub_sub_collection_recall_df["Level Type"] = "Sub_Sub_Collection"
digitalhub_sub_sub_collection_recall_df["Level Number"] = "3"
digitalhub_sub_sub_collection_recall_df['Level Number'] = digitalhub_sub_sub_collection_recall_df['Level Number'].apply(int)

## Create a column from the index
digitalhub_sub_sub_collection_recall_df['sub_sub_collection_rowid'] = digitalhub_sub_sub_collection_recall_df.index

## Create a new column called Sub_Collection_ID
digitalhub_sub_sub_collection_recall_df['Sub_Sub_Collection_ID'] = digitalhub_sub_sub_collection_recall_df['Id']

## Checked: No problems


In [59]:
## Inspect the "members" column and create a list of member IDs

row_sub_sub_collection_member_list = []
column_sub_sub_collection_member_dict ={}
count_sub_sub_collection_member_dict = {}

for k, v in digitalhub_sub_sub_collection_recall_df["members"].items():
    for value in v: 
        member = value["Id"]
        row_sub_sub_collection_member_list.append(member)
        count_sub_sub_collection_member_dict[k] = len(row_sub_sub_collection_member_list)
    column_sub_sub_collection_member_dict[k] = row_sub_sub_collection_member_list
    row_sub_sub_collection_member_list =[]

## Append the column member dictionary to the DigitalHub Community DF dataframe

digitalhub_sub_sub_collection_recall_df['Member_List'] =digitalhub_sub_sub_collection_recall_df.index.map(column_sub_sub_collection_member_dict)
digitalhub_sub_sub_collection_recall_df['Member_List_Count'] =digitalhub_sub_sub_collection_recall_df.index.map(count_sub_sub_collection_member_dict)
digitalhub_sub_collection_recall_df.head()

## Checked: No problems

,Multi-page?,Title,Keyword,Resource type(s),Rights,Creator,Contributor,Description,Abstract,Original Bibliographic Citation,...,Digital Origin,Id,uri,members,Level Type,Level Number,sub_collection_rowid,Sub_Collection_ID,Member_List,Member_List_Count
0,False,Fall 2017,"[medical research, Biostatistics Collaboration...",[],[All rights reserved],[],[],Free introductory lectures on biostatistics in...,[],[],...,[],17c76dde-4be6-4d2d-9020-3904a9451617,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Biostatistics 101: the minor deta...,Sub_Collection,2,0,17c76dde-4be6-4d2d-9020-3904a9451617,"[31fb931a-2e6b-4681-b9e8-63c70346ffc3, 36214e9...",5.0
1,False,Winter 2019,"[Biostatistics Collaboration Center , medical ...",[],[All rights reserved],[],[],Free introductory lectures on biostatistics in...,[],[],...,[],5d8b5f8e-c46b-4e97-a5a2-9c118c9cca24,https://digitalhub.northwestern.edu/collection...,[{'Title': ['A Statistician's Guide to REDCap'...,Sub_Collection,2,1,5d8b5f8e-c46b-4e97-a5a2-9c118c9cca24,"[31dabd3f-f8ea-4229-8f7d-ea9fc701d993, a31212f...",5.0
2,False,Fall 2016,"[Biostatistics Collaboration Center , biostati...",[],[All rights reserved],[],[],Free introductory lectures on biostatistics in...,[],[],...,[],fc389d13-2430-409b-82fd-a4b26613d350,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Clinical Trials: Highlights from ...,Sub_Collection,2,2,fc389d13-2430-409b-82fd-a4b26613d350,"[759b7521-06d4-4676-847f-4e9496d46880, 02490b1...",7.0
3,True,2019-2020,"[Biostatistics Collaboration Center, Statistic...",[],[All rights reserved],[],[],Free introductory lectures on biostatistics in...,[],[],...,[],a86e1412-d72c-4cae-b8ca-16fd834cb128,https://digitalhub.northwestern.edu/collection...,[{'Title': ['Logistic Regression: Odds & Ends'...,Sub_Collection,2,3,a86e1412-d72c-4cae-b8ca-16fd834cb128,"[9ba34043-72e7-4e87-b872-184274f256df, ceab6fe...",5.0
4,None,Chicago Biomedical Informatics and Data Scienc...,"[NUCATS, CDSI, informatics, data science]",[],[],[],[],Biomedical and Health Informatics and Data Sci...,[],[],...,[],6d66813c-a6e8-49b0-a568-d9378f82da51,https://digitalhub.northwestern.edu/collection...,[],Sub_Collection,2,4,6d66813c-a6e8-49b0-a568-d9378f82da51,[],NaN


In [60]:
## Export file to excel, without the Pandas index, but with the header

digitalhub_sub_sub_collection_recall_df.to_excel("outputs/digitalhub_sub_sub_collection_recall_df.xlsx", index=False, header=True)

## Checked: No problems

###### Split Recall Sub_Sub_Collection into Members

In [61]:
## Create a dataframe for each members row and concatenate all of these into one member_df

sub_sub_collection_members_dfs_list = []

for k, v in digitalhub_sub_sub_collection_recall_df["members"].items():
#     print("this is k: ", k)
#     print("this is v: ", v)
    sub_sub_collection_member_df = pd.json_normalize(v)
    sub_sub_collection_member_df['sub_sub_collection_rowid'] = k
    sub_sub_collection_members_dfs_list.append(sub_sub_collection_member_df)

digitalhub_sub_sub_collection_members_df = pd.concat(sub_sub_collection_members_dfs_list, sort=False).reset_index(drop='index')
digitalhub_sub_sub_collection_members_df.head()

## Resource
## https://stackoverflow.com/questions/62816027/convert-pandas-json-column-to-multiple-rows

## Checked: No problems

,Title,Resource type(s),Keyword,Rights,Creator,Contributor,Description,Abstract,Original Bibliographic Citation,Related URL,...,Acknowledgments,Grants And Funding,DOI,ARK,Id,File Size,File Format,uri,download,sub_sub_collection_rowid
0,[ Portrait of James A. Conner],[Photographs],[12th General Hospital],[http://creativecommons.org/publicdomain/mark/...,[Creator not identified.],None,"[Portrait of James A. Conner in suit and tie, ...",None,None,None,...,None,None,[doi:10.18131/g3-j643-be88],None,d2dfa198-1da0-4b8f-9d28-c70f786f89bb,650570.0,[jpeg (JPEG File Interchange Format)],https://digitalhub.northwestern.edu/files/d2df...,https://digitalhub.northwestern.edu/downloads/...,0
1,[James A. Conner and Classmates],[Photographs],"[Medical School, Northwestern University]",[All rights reserved],[Creator not identified.],None,"[(Left to Right): ""George, Goodfrriend, Bebris...",None,None,None,...,None,None,[doi:10.18131/g3-2ksd-xv27],None,a5813d88-5b10-4be4-953e-aa81502c0aa3,1399972.0,[tiff (Tagged Image File Format)],https://digitalhub.northwestern.edu/files/a581...,https://digitalhub.northwestern.edu/downloads/...,0
2,[James A. Conner Hunting],[Photographs],[Hunting],[All rights reserved],[Almer Coe & Company],None,[Mounted photograph of James A. Conner on skis...,None,None,None,...,None,None,[doi:10.18131/g3-nzhr-9r28],None,5ee1c07c-3c06-4652-b423-4149baaadfa8,11578146.0,[tiff (Tagged Image File Format)],https://digitalhub.northwestern.edu/files/5ee1...,https://digitalhub.northwestern.edu/downloads/...,0
3,[Portrait of James A. Conner in Uniform],[Photographs],"[12th General Hospital, World War II]",[http://creativecommons.org/publicdomain/mark/...,[Creator not identified.],None,None,None,None,None,...,None,None,[doi:10.18131/g3-h7z5-y289],None,3dc862fd-d838-4e79-b7db-adda7f9b05e4,580278.0,[jpeg (JPEG File Interchange Format)],https://digitalhub.northwestern.edu/files/3dc8...,https://digitalhub.northwestern.edu/downloads/...,0
4,[WWII U.S. Army Eagle Officer Hat Pin],[Other],"[Insignia, Uniform, 12th General Hospital, Wor...",[http://creativecommons.org/publicdomain/mark/...,[United States. Army],None,[World War II U.S. Army eagle officer hat pin ...,None,None,None,...,None,None,[doi:10.18131/g3-26yx-3b95],None,84698374-b4ae-43fe-bd1e-3b4c3d39772a,1340772.0,[jpeg (JPEG File Interchange Format)],https://digitalhub.northwestern.edu/files/8469...,https://digitalhub.northwestern.edu/downloads/...,1


In [62]:
## Add Column to digitalhub_community_df to indicate that these results are a collection

# digitalhub_sub_collection_df["Level"] =  "Collection"

digitalhub_sub_sub_collection_members_df['Level Type'] = np.where(digitalhub_sub_sub_collection_members_df['DOI'].isnull(),"Sub_Sub_Sub_Collection", "Record" )

digitalhub_sub_sub_collection_members_df['Level Number'] = np.where(digitalhub_sub_sub_collection_members_df['DOI'].isnull(),"4", "6" )
digitalhub_sub_sub_collection_members_df['Level Number'] = digitalhub_sub_sub_collection_members_df['Level Number'].apply(int)
digitalhub_sub_sub_collection_members_df.head()

## Checked: No problems

,Title,Resource type(s),Keyword,Rights,Creator,Contributor,Description,Abstract,Original Bibliographic Citation,Related URL,...,DOI,ARK,Id,File Size,File Format,uri,download,sub_sub_collection_rowid,Level Type,Level Number
0,[ Portrait of James A. Conner],[Photographs],[12th General Hospital],[http://creativecommons.org/publicdomain/mark/...,[Creator not identified.],None,"[Portrait of James A. Conner in suit and tie, ...",None,None,None,...,[doi:10.18131/g3-j643-be88],None,d2dfa198-1da0-4b8f-9d28-c70f786f89bb,650570.0,[jpeg (JPEG File Interchange Format)],https://digitalhub.northwestern.edu/files/d2df...,https://digitalhub.northwestern.edu/downloads/...,0,Record,6
1,[James A. Conner and Classmates],[Photographs],"[Medical School, Northwestern University]",[All rights reserved],[Creator not identified.],None,"[(Left to Right): ""George, Goodfrriend, Bebris...",None,None,None,...,[doi:10.18131/g3-2ksd-xv27],None,a5813d88-5b10-4be4-953e-aa81502c0aa3,1399972.0,[tiff (Tagged Image File Format)],https://digitalhub.northwestern.edu/files/a581...,https://digitalhub.northwestern.edu/downloads/...,0,Record,6
2,[James A. Conner Hunting],[Photographs],[Hunting],[All rights reserved],[Almer Coe & Company],None,[Mounted photograph of James A. Conner on skis...,None,None,None,...,[doi:10.18131/g3-nzhr-9r28],None,5ee1c07c-3c06-4652-b423-4149baaadfa8,11578146.0,[tiff (Tagged Image File Format)],https://digitalhub.northwestern.edu/files/5ee1...,https://digitalhub.northwestern.edu/downloads/...,0,Record,6
3,[Portrait of James A. Conner in Uniform],[Photographs],"[12th General Hospital, World War II]",[http://creativecommons.org/publicdomain/mark/...,[Creator not identified.],None,None,None,None,None,...,[doi:10.18131/g3-h7z5-y289],None,3dc862fd-d838-4e79-b7db-adda7f9b05e4,580278.0,[jpeg (JPEG File Interchange Format)],https://digitalhub.northwestern.edu/files/3dc8...,https://digitalhub.northwestern.edu/downloads/...,0,Record,6
4,[WWII U.S. Army Eagle Officer Hat Pin],[Other],"[Insignia, Uniform, 12th General Hospital, Wor...",[http://creativecommons.org/publicdomain/mark/...,[United States. Army],None,[World War II U.S. Army eagle officer hat pin ...,None,None,None,...,[doi:10.18131/g3-26yx-3b95],None,84698374-b4ae-43fe-bd1e-3b4c3d39772a,1340772.0,[jpeg (JPEG File Interchange Format)],https://digitalhub.northwestern.edu/files/8469...,https://digitalhub.northwestern.edu/downloads/...,1,Record,6


##### Concatenate Recall Sub_Sub_Collection with Members

In [63]:
## Concatenate the DigitalHub Sub_Sub_Collection_Recall Dataframe using the 'rowid' to the DigitalHub Sub_Sub_Collection Members Dataframe using the 'rowid'

digitalhub_sub_sub_col_members_df = pd.concat([digitalhub_sub_sub_collection_recall_df, digitalhub_sub_sub_collection_members_df], axis=0).sort_values(by=['sub_sub_collection_rowid'])
digitalhub_sub_sub_col_members_df.head()

## Checked: No problems

,Multi-page?,Title,Keyword,Resource type(s),Rights,Creator,Contributor,Description,Abstract,Original Bibliographic Citation,...,Member_List,Member_List_Count,Page Number,Acknowledgments,Grants And Funding,DOI,ARK,File Size,File Format,download
0,False,Conner Photographs,"[12th General Hospital, World War II]",[],[],[Creator not identified],[],Photographs of James A. Conner.,[],[],...,"[d2dfa198-1da0-4b8f-9d28-c70f786f89bb, a5813d8...",4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,[ Portrait of James A. Conner],[12th General Hospital],[Photographs],[http://creativecommons.org/publicdomain/mark/...,[Creator not identified.],None,"[Portrait of James A. Conner in suit and tie, ...",None,None,...,NaN,NaN,None,None,None,[doi:10.18131/g3-j643-be88],None,650570.0,[jpeg (JPEG File Interchange Format)],https://digitalhub.northwestern.edu/downloads/...
1,NaN,[James A. Conner and Classmates],"[Medical School, Northwestern University]",[Photographs],[All rights reserved],[Creator not identified.],None,"[(Left to Right): ""George, Goodfrriend, Bebris...",None,None,...,NaN,NaN,None,None,None,[doi:10.18131/g3-2ksd-xv27],None,1399972.0,[tiff (Tagged Image File Format)],https://digitalhub.northwestern.edu/downloads/...
2,NaN,[James A. Conner Hunting],[Hunting],[Photographs],[All rights reserved],[Almer Coe & Company],None,[Mounted photograph of James A. Conner on skis...,None,None,...,NaN,NaN,None,None,None,[doi:10.18131/g3-nzhr-9r28],None,11578146.0,[tiff (Tagged Image File Format)],https://digitalhub.northwestern.edu/downloads/...
3,NaN,[Portrait of James A. Conner in Uniform],"[12th General Hospital, World War II]",[Photographs],[http://creativecommons.org/publicdomain/mark/...,[Creator not identified.],None,None,None,None,...,NaN,NaN,None,None,None,[doi:10.18131/g3-h7z5-y289],None,580278.0,[jpeg (JPEG File Interchange Format)],https://digitalhub.northwestern.edu/downloads/...


In [64]:
## Export file to excel, without the Pandas index, but with the header
digitalhub_sub_sub_col_members_df.to_excel("outputs/digitalhub_sub_sub_col_members_df.xlsx", index=False, header=True)

## Checked: No problems

In [65]:
## Sort the dataframe by sub_sub_collection_rowid and Level Number to prepare for group by and forward fill of Sub_Sub_Collection_ID
## sub_sub_collection_rowid then Level Number

digitalhub_sub_sub_col_members_df.sort_values(by = ['sub_sub_collection_rowid','Level Number'], ascending = [True, True], inplace=True)

## Checked: No problems

In [66]:
## Groupby Sub_Sub_Collection_rowid and fill forward the Sub_Sub_Collection_ID into members of sub_sub_collections (i.e. sub_sub_sub_collections and records)

digitalhub_sub_sub_col_members_df.update(digitalhub_sub_sub_col_members_df.groupby(['sub_sub_collection_rowid'])['Sub_Sub_Collection_ID'].ffill())

## Resources
## https://stackoverflow.com/questions/64795941/how-do-i-forward-fill-nas-with-condition-of-2-other-cells-being-equal-in-pandas
## https://stackoverflow.com/questions/27012151/forward-fill-specific-columns-in-pandas-dataframe


In [67]:
## Create a column for Sub_Sub_Sub_Collection_ID to hold the Id from columns with Level Type as "Sub_Sub_Sub_Collection"

digitalhub_sub_sub_col_members_df['Sub_Sub_Sub_Collection_ID'] = np.where(digitalhub_sub_sub_col_members_df['Level Type'] == 'Sub_Sub_Sub_Collection',digitalhub_sub_sub_col_members_df['Id'], np.nan)

## Resources
## https://stackoverflow.com/questions/67043249/how-to-use-np-where-in-creating-new-column-using-previous-rows

## Checked: No problems

In [68]:
## Export file to excel, without the Pandas index with the header

digitalhub_sub_sub_col_members_df.to_excel("outputs/digitalhub_sub_sub_col_members_df.xlsx", header=True)

## Checked: No problems

##### Concatenate Sub_Sub_Collection with Sub_Collection and Community

In [69]:
## Concatenate the DigitalHub digitalhub_sub_col_sub_col_df Dataframe using the 'Id' to the DigitalHub digitalhub_sub_sub_col_members_df Dataframe using the 'Id'
#digitalhub_sub_col_sub_col_df
#digitalhub_sub_sub_col_members_df

digitalhub_sub_col_sub_col_sub_col_df = pd.concat([digitalhub_sub_col_sub_col_df, digitalhub_sub_sub_col_members_df], axis=0).sort_values(by=['Community_ID', 'Sub_Collection_ID'])
digitalhub_sub_col_sub_col_sub_col_df.head()

## Note: Creates duplicate rows for "Sub_Sub_Collection_ID"
## Checked: No problems

,index,Id,Multi-page?,Title,Keyword,Resource type(s),Rights,Creator,Contributor,Description,...,DOI,ARK,File Size,File Format,download,Sub_Collection_ID,sub_collection_rowid,Sub_Sub_Collection_ID,sub_sub_collection_rowid,Sub_Sub_Sub_Collection_ID
30,30.0,074eef38-f1fa-48bb-aa1a-dd895b73d2b1,None,Northwestern Pepper Center,"[primary care, geriatrics, aging research, mul...",[],[],[],[],The mission of the Northwestern Pepper Center ...,...,None,None,NaN,None,None,None,NaN,None,NaN,NaN
32,32.0,07b25bee-4a47-466a-b9b8-70d7a392fab0,False,12th General Hospital Collection,[World War II],[],[],"[Connor, James A., Mason, Michael L., 1895-1963]",[],The 12th General Hospital Collection is compri...,...,None,None,NaN,None,None,07b25bee-4a47-466a-b9b8-70d7a392fab0,9.0,None,NaN,NaN
683,683.0,a97853c7-82d3-408c-95d8-c72a04acb41f,False,[James A. Conner],"[World War II, Northwestern University]",[],[],[],[],,...,None,None,NaN,None,None,a97853c7-82d3-408c-95d8-c72a04acb41f,51.0,a97853c7-82d3-408c-95d8-c72a04acb41f,NaN,NaN
70,70.0,102d9812-55fa-4aeb-b910-04e534f7a7ef,False,[Conner Photographs],"[12th General Hospital, World War II]",None,None,[Creator not identified],None,[Photographs of James A. Conner.],...,None,None,NaN,None,None,a97853c7-82d3-408c-95d8-c72a04acb41f,51.0,102d9812-55fa-4aeb-b910-04e534f7a7ef,NaN,NaN
508,508.0,7b81356b-6eaa-4570-bd18-355714700aed,False,[Conner Insignia],"[12th General Hospital, World War II]",None,[http://creativecommons.org/publicdomain/mark/...,[United States. Army],None,[This Sub-collection contains insignia and bad...,...,None,None,NaN,None,None,a97853c7-82d3-408c-95d8-c72a04acb41f,51.0,7b81356b-6eaa-4570-bd18-355714700aed,NaN,NaN


In [70]:
## Export file to excel, without the Pandas index with the header

digitalhub_sub_col_sub_col_sub_col_df.to_excel("outputs/digitalhub_sub_col_sub_col_sub_col_df.xlsx", header=True)

## Checked: No problems

In [71]:
## Sort the dataframe by Id and Level Number to get ready to groupby ID and remove duplicate Sub_Sub_Collections from the concatenate
digitalhub_sub_col_sub_col_sub_col_df.sort_values(by = ['Id','Level Number'], ascending = [True, True], inplace=True)

## Checked: No problems

In [72]:
## Remove duplicates for Sub_Sub_Collection ID between concatenated dataframes
digitalhub_sub_col_sub_col_sub_col_df = digitalhub_sub_col_sub_col_sub_col_df.groupby(['Id'], as_index=False).first().reset_index()

## Checked: No problems

In [73]:
## Sort the dataframe by Sub_Sub_Collection_ID and then Level Number to get ready to fill Community IDs into connected Sub_Sub_Collections

digitalhub_sub_col_sub_col_sub_col_df.sort_values(by = ['Sub_Sub_Collection_ID','Level Number'], ascending = [True, True], inplace=True)

## Checked: No problems

In [74]:
## Fill foward the Community_ID into Records based on their Sub_Sub_Collection_IDs

digitalhub_sub_col_sub_col_sub_col_df.update(digitalhub_sub_col_sub_col_sub_col_df.groupby(['Sub_Sub_Collection_ID'])['Community_ID'].ffill())

## Other options
##digitalhub_sub_col_sub_col_df['Community_ID'] = digitalhub_sub_col_sub_col_df.groupby(['Sub_Collection_ID'])['Community_ID'].fillna(method='ffill')
##digitalhub_sub_col_sub_col_df['Community_ID'] = digitalhub_sub_col_sub_col_df.groupby(['Sub_Collection_ID'])['Community_ID'].transform(lambda x: x.ffill())
##digitalhub_sub_col_sub_col_df['Community_ID'] = digitalhub_sub_col_sub_col_df.groupby(['Sub_Collection_ID'])['Community_ID'].fillna(method='ffill')

## Resources
## https://stackoverflow.com/questions/64795941/how-do-i-forward-fill-nas-with-condition-of-2-other-cells-being-equal-in-pandas
## https://stackoverflow.com/questions/58181262/groupby-with-ffill-deletes-group-and-does-not-put-group-in-index

## Checked: No problems

In [75]:
## Export file to excel with the header

digitalhub_sub_col_sub_col_sub_col_df.to_excel("outputs/digitalhub_sub_col_sub_col_sub_col_df.xlsx",  header=True)

## Checked: No problems

#### Recal Sub_Sub_Sub_Collections
##### Do no need if no sub_sub_sub_collections

In [76]:
## DO NOT NEED if there are no sub_sub_sub_collections

## Extract a the Series for sub_sub_sub_collection "dh_id" and transform into a list

digitalhub_sub_sub_sub_collection_series = digitalhub_sub_col_sub_col_sub_col_df[digitalhub_sub_col_sub_col_sub_col_df["Level Type"] == "Sub_Sub_Sub_Collection"]['Id']
digitalhub_sub_sub_sub_collection_list = digitalhub_sub_sub_sub_collection_series.tolist()

print(digitalhub_sub_sub_sub_collection_list)

## Checked: No problems 

[]


In [77]:
## DO NOT NEED if there are no sub_sub_sub_collections

## Loop through list of DigitalHub Sub_Sub_Sub_Collections and use the urllib.request to get the json data
## test_list = ['2cc92425-b656-47ea-a3b4-825405ee6088', 'a86e1412-d72c-4cae-b8ca-16fd834cb128','fc389d13-2430-409b-82fd-a4b26613d350']

# multi_digitalhub_sub_sub_sub_collection_list = []
# digitalhub_sub_sub_sub_collection_problem_list = []

# for item in digitalhub_sub_sub_sub_collection_list:
#     try:

#         with urllib.request.urlopen(f"https://digitalhub.northwestern.edu/collections/{item}.json" ) as url:
#             single_digitalhub_sub_sub_sub_collection_dict = json.loads(url.read().decode())
#             multi_digitalhub_sub_sub_sub_collection_list.append(single_digitalhub_sub_sub_sub_collection_dict)
#             print(item)
#         time.sleep(1)
       

#     except urllib.error.HTTPError as http_err:
#         print(item)
#         digitalhub_sub_sub_sub_collection_problem_list.append(item)
#         print(f'HTTP error occurred: {http_err}')  # Python 3.6
        

#     except urllib.error.URLError as url_err:
#         print(item)
#         digitalhub_sub_sub_sub_collection_problem_list.append(item)
#         print(f'URL error occurred: {url_err}. ')  # Python 3.6
       

#     except json.JSONDecodeError as json_err:
#         print(item)
#         digitalhub_sub_sub_sub_collection_problem_list.append(item)
#         print(f'JSON Decode error occurred: {json_err}. ', 'Poorly formed JSON.')  # Python 3.6
       
        
#     except Exception as err:
#         print(item)
#         digitalhub_sub_sub_sub_collection_problem_list.append(item)
#         print(f'Other error occurred: {err}. ')  # Python 3.6
        
              
#     else:
#         print('Success!')



## Resources
## https://docs.python.org/3/library/urllib.request.html

## Checked: No problems

In [78]:
## DO NOT NEED if there are no sub_sub_sub_collections

## Inspect the results of the URL Query for DigitalHub Collections that will become communities in Prism

# print(multi_digitalhub_sub_sub_sub_collection_list)
# print(digitalhub_sub_sub_sub_collection_problem_list)

In [79]:
## DO NOT NEED if there are no sub_sub_sub_collections
## Create a dataframe from DigitalHub json for DigitalHub Sub_Sub_Sub_Collections

# digitalhub_sub_sub_sub_collection_recall_df = pd.DataFrame.from_dict(json_normalize(multi_digitalhub_sub_sub_sub_collection_list, max_level=1))
# digitalhub_sub_sub_sub_collection_recall_df.head(10)

In [80]:
## DO NOT NEED if there are no sub_sub_sub_collections
### DO NOT NEED if there are no problems with sub_sub_sub_collections

## Upload .txt files of the problem Sub_Sub_Sub_Collection json metadata

# name_here_path = r"data\sub_sub_sub_collection\NAME HERE.txt"
# problem_sub_sub_sub_dict_list = []
# path_sub_sub_list = [name_here_path]

# for path in path_sub_sub_sub_list: 
#     with open(path, 'r', encoding= 'utf8') as f:
#         s = f.read()
#         while True:
#             try:
#                 result = json.loads(s)   # try to parse...
#                 break                    # parsing worked -> exit loop
#             except Exception as e:
#                 # "Expecting , delimiter: line 34 column 54 (char 1158)"
#                 # position of unexpected character after '"'
#                 unexp = int(re.findall(r'\(char (\d+)\)', str(e))[0])
#                 # position of unescaped '"' before that
#                 unesc = s.rfind(r'"', 0, unexp)
#                 s = s[:unesc] + r'\"' + s[unesc+1:]
#                 # position of correspondig closing '"' (+2 for inserted '\')
#                 closg = s.find(r'"', unesc + 2)
#                 s = s[:closg] + r'\"' + s[closg+1:]
# #         print(result)
#         problem_sub_sub_sub_dict_list.append(result)



## https://stackoverflow.com/questions/16573332/jsondecodeerror-expecting-value-line-1-column-1-char-0
## https://stackoverflow.com/questions/18514910/how-do-i-automatically-fix-an-invalid-json-string

## Checked: No problems

In [81]:
## DO NOT NEED if there are no sub_sub_sub_collections
### DO NOT NEED if there are no problems with sub_sub_sub_collections

# problem_sub_sub_sub_df = pd.DataFrame.from_dict(json_normalize(problem_sub_sub_sub_dict_list, max_level=1))
# problem_sub_sub_sub_df.head()

## Checked: No problems

In [82]:
## DO NOT NEED if there are no sub_sub_sub_collections
### DO NOT NEED if there are no problems with sub_sub_sub_collections

## Concatenate the DigitalHub Community Dataframe to the problem_df

# digitalhub_sub_sub_sub_collection_recall_df= pd.concat([digitalhub_sub_sub_sub_collection_recall_df, problem_sub_sub_sub_df], axis=0)
# digitalhub_sub_sub_sub_collection_recall_df.reset_index(inplace=True, drop=True) 
# digitalhub_sub_sub_sub_collection_recall_df.head(10)

## Checked: No problems


In [83]:
# ## DO NOT NEED if there are no sub_sub_sub_collections

# ## Add Column to digitalhub_sub_sub_sub_collection_recall_df to indicate that these results are a sub_sub_sub_collection
# digitalhub_sub_sub_sub_collection_recall_df["Level Type"] = "Sub_Sub_Sub_Collection"
# digitalhub_sub_sub_sub_collection_recall_df["Level Number"] = "4"
# digitalhub_sub_sub_sub_collection_recall_df['Level Number'] = digitalhub_sub_sub_sub_collection_recall_df['Level Number'].apply(int)

# ## Create a column from the index
# digitalhub_sub_sub_sub_collection_recall_df['sub_sub_sub_collection_rowid'] = digitalhub_sub_sub_sub_collection_recall_df.index

# ## Create a new column called Sub_Collection_ID
# digitalhub_sub_sub_sub_collection_recall_df['Sub_Sub_Sub_Collection_ID'] = digitalhub_sub_sub_sub_collection_recall_df['Id']

# ## Checked: No problems

In [84]:
# ## DO NOT NEED if there are no sub_sub_sub_collections
# ## Inspect the "members" column and create a list of member IDs

# row_sub_sub_sub_collection_member_list = []
# column_sub_sub_sub_collection_member_dict ={}
# count_sub_sub_sub_collection_member_dict = {}

# for k, v in digitalhub_sub_sub_sub_collection_recall_df["members"].items():
#     for value in v: 
#         member = value["Id"]
#         row_sub_sub_sub_collection_member_list.append(member)
#         count_sub_sub_sub_collection_member_dict[k] = len(row_sub_sub_sub_collection_member_list)
#     column_sub_sub_sub_collection_member_dict[k] = row_sub_sub_sub_collection_member_list
#     row_sub_sub_sub_collection_member_list =[]

# ## Append the column member dictionary to the DigitalHub Community DF dataframe

# digitalhub_sub_sub_sub_collection_recall_df['Member_List'] =digitalhub_sub_sub_sub_collection_recall_df.index.map(column_sub_sub_sub_collection_member_dict)
# digitalhub_sub_sub_sub_collection_recall_df['Member_List_Count'] =digitalhub_sub_sub_sub_collection_recall_df.index.map(count_sub_sub_sub_collection_member_dict)
# digitalhub_sub_sub_collection_recall_df.head()

# ## Checked: No problems

In [85]:
# ## DO NOT NEED if there are no sub_sub_sub_collections
# ## Export file to excel, without the Pandas index with the header

# digitalhub_sub_sub_sub_collection_recall_df.to_excel("outputs/digitalhub_sub_sub_sub_collection_recall_df.xlsx", header=True)

# ## Checked: No problems

##### Split the Recall Sub_Sub_Sub_Collection into Members

In [86]:
## DO NOT NEED if there are no sub_sub_sub_collections
## Create a dataframe for each members row and concatenate all of these into one member_df

# sub_sub_sub_collection_members_dfs_list = []

# for k, v in digitalhub_sub_sub_sub_collection_recall_df["members"].items():
# #     print("this is k: ", k)
# #     print("this is v: ", v)
#     sub_sub_sub_collection_member_df = pd.json_normalize(v)
#     sub_sub_sub_collection_member_df['sub_sub_sub_collection_rowid'] = k
#     sub_sub_sub_collection_members_dfs_list.append(sub_sub_sub_collection_member_df)

# digitalhub_sub_sub_sub_collection_members_df = pd.concat(sub_sub_sub_collection_members_dfs_list, sort=False).reset_index(drop='index')
# digitalhub_sub_sub_sub_collection_members_df.head()

# ## Resource
# ## https://stackoverflow.com/questions/62816027/convert-pandas-json-column-to-multiple-rows

# ## Checked: No problems

In [87]:
## DO NOT NEED if there are no sub_sub_sub_collections
## Add Column to digitalhub_community_df to indicate that these results are a collection

# digitalhub_sub_collection_df["Level"] =  "Collection"

# digitalhub_sub_sub_sub_collection_members_df['Level Type'] = np.where(digitalhub_sub_sub_sub_collection_members_df['DOI'].isnull(),"Sub_Sub_Sub_Sub_Collection", "Record" )

# digitalhub_sub_sub_sub_collection_members_df['Level Number'] = np.where(digitalhub_sub_sub_sub_collection_members_df['DOI'].isnull(),"5", "6" )
# digitalhub_sub_sub_sub_collection_members_df['Level Number'] = digitalhub_sub_sub_sub_collection_members_df['Level Number'].apply(int)
# digitalhub_sub_sub_sub_collection_members_df.head()

## Checked: No problems

##### Concatenate Reacal Sub_Sub_Sub_Collection with Members

In [88]:
# ## DO NOT NEED if there are no sub_sub_sub_collections
# ## Concatenate the DigitalHub Sub_Sub_Sub_Collection_Recall Dataframe using the 'rowid' to the DigitalHub Sub_Sub_Sub_Collection Members Dataframe using the 'rowid'

# digitalhub_sub_sub_sub_col_members_df = pd.concat([digitalhub_sub_sub_sub_collection_recall_df, digitalhub_sub_sub_sub_collection_members_df], axis=0).sort_values(by=['sub_sub_sub_collection_rowid'])
# digitalhub_sub_sub_sub_col_members_df.head()

# ## Checked: No problems

In [89]:
# ## DO NOT NEED if there are no sub_sub_sub_collections
# ## Export file to excel, without the Pandas index, but with the header
# digitalhub_sub_sub_sub_col_members_df.to_excel("outputs/digitalhub_sub_sub_sub_col_members_df.xlsx", index=False, header=True)

# ## Checked: No problems

In [90]:
## DO NOT NEED if there are no sub_sub_sub_collections
# ## Sort the dataframe by sub_sub_sub_collection_rowid and Level Number to prepare for group by and forward fill of Sub_Sub_Sub_Collection_ID
# ## sub_sub_sub_collection_rowid then Level Number

# digitalhub_sub_sub_sub_col_members_df.sort_values(by = ['sub_sub_sub_collection_rowid','Level Number'], ascending = [True, True], inplace=True)

# ## Checked: No problems

In [91]:
# ## DO NOT NEED if there are no sub_sub_sub_collections
# ## Groupby Sub_Sub_Sub_Collection_rowid and fill forward the Sub_Sub_Sub_Collection_ID into members of sub_sub_sub_collections (i.e. sub_sub_sub_sub_collections and records)

# digitalhub_sub_sub_sub_col_members_df.update(digitalhub_sub_sub_sub_col_members_df.groupby(['sub_sub_sub_collection_rowid'])['Sub_Sub_Sub_Collection_ID'].ffill())

# ## Resources
# ## https://stackoverflow.com/questions/64795941/how-do-i-forward-fill-nas-with-condition-of-2-other-cells-being-equal-in-pandas
# ## https://stackoverflow.com/questions/27012151/forward-fill-specific-columns-in-pandas-dataframe


In [92]:
# ## DO NOT NEED if there are no sub_sub_sub_collections
# ## Create a column for Sub_Sub_Sub_Sub_Collection_ID to hold the Id from columns with Level Type as "Sub_Sub_Sub_Sub_Collection"

# digitalhub_sub_sub_sub_col_members_df['Sub_Sub_Sub_Sub_Collection_ID'] = np.where(digitalhub_sub_sub_sub_col_members_df['Level Type'] == 'Sub_Sub_Sub_Sub_Collection',digitalhub_sub_sub_sub_col_members_df['Id'], np.nan)

# ## Resources
# ## https://stackoverflow.com/questions/67043249/how-to-use-np-where-in-creating-new-column-using-previous-rows

# ## Checked: No problems

In [93]:
# ## DO NOT NEED if there are no sub_sub_sub_collections
# ## Export file to excel, without the Pandas index with the header

# digitalhub_sub_sub_sub_col_members_df.to_excel("outputs/digitalhub_sub_sub_sub_col_members_df.xlsx", header=True)

# ## Checked: No problems

##### Concetenate Sub_Sub_Sub_Sub_Collection with Sub_Sub_Collection, Sub_Collection, and Community

In [94]:
# ## DO NOT NEED if there are no sub_sub_sub_collections
# ## Concatenate the DigitalHub digitalhub_sub_col_sub_col_df Dataframe using the 'Id' to the DigitalHub digitalhub_sub_sub_col_members_df Dataframe using the 'Id'
# #digitalhub_sub_col_sub_col_df
# #digitalhub_sub_sub_col_members_df

# digitalhub_sub_col_sub_col_sub_col_sub_col_df = pd.concat([digitalhub_sub_col_sub_col_sub_col_df, digitalhub_sub_sub_sub_col_members_df], axis=0).sort_values(by=['Community_ID', 'Sub_Collection_ID'])
# digitalhub_sub_col_sub_col_sub_col_sub_col_df.head()

# ## Note: Creates duplicate rows for "Sub_Sub_Collection_ID"
# ## Checked: No problems

In [95]:
# ## DO NOT NEED if there are no sub_sub_sub_collections
# ## Sort the dataframe by Id and Level Number to get ready to groupby ID and remove duplicate Sub_Sub_Sub_Collections from the concatenate
# digitalhub_sub_col_sub_col_sub_col_sub_col_df.sort_values(by = ['Id','Level Number'], ascending = [True, True], inplace=True)

# ## Checked: No problems

In [96]:
# ## DO NOT NEED if there are no sub_sub_sub_collections
# digitalhub_sub_col_sub_col_sub_col_sub_col_df.drop(columns=['level_0'], inplace=True)

In [97]:
# ## DO NOT NEED if there are no sub_sub_sub_collections
# ## Remove duplicates for Sub_Sub_Sub_Collection ID between concatenated dataframes
# digitalhub_sub_col_sub_col_sub_col_sub_col_df = digitalhub_sub_col_sub_col_sub_col_sub_col_df.groupby(['Id'], as_index=False).first().reset_index()

# ## Checked: No problems

In [98]:
# ## DO NOT NEED if there are no sub_sub_sub_collections
# ## Sort the dataframe by Sub_Sub_Sub_Collection_ID and then Level Number to get ready to fill Community IDs into connected Sub_Sub_Sub_Collections

# digitalhub_sub_col_sub_col_sub_col_sub_col_df.sort_values(by = ['Sub_Sub_Sub_Collection_ID','Level Number'], ascending = [True, True], inplace=True)

# ## Checked: No problems

In [99]:
# ## DO NOT NEED if there are no sub_sub_sub_collections
# ## Fill foward the Community_ID into Records based on their Sub_Sub_Sub_Collection_IDs

# digitalhub_sub_col_sub_col_sub_col_sub_col_df.update(digitalhub_sub_col_sub_col_sub_col_sub_col_df.groupby(['Sub_Sub_Sub_Collection_ID'])['Community_ID'].ffill())

# ## Other options
# ##digitalhub_sub_col_sub_col_df['Community_ID'] = digitalhub_sub_col_sub_col_df.groupby(['Sub_Collection_ID'])['Community_ID'].fillna(method='ffill')
# ##digitalhub_sub_col_sub_col_df['Community_ID'] = digitalhub_sub_col_sub_col_df.groupby(['Sub_Collection_ID'])['Community_ID'].transform(lambda x: x.ffill())
# ##digitalhub_sub_col_sub_col_df['Community_ID'] = digitalhub_sub_col_sub_col_df.groupby(['Sub_Collection_ID'])['Community_ID'].fillna(method='ffill')

# ## Resources
# ## https://stackoverflow.com/questions/64795941/how-do-i-forward-fill-nas-with-condition-of-2-other-cells-being-equal-in-pandas
# ## https://stackoverflow.com/questions/58181262/groupby-with-ffill-deletes-group-and-does-not-put-group-in-index

# ## Checked: No problems

In [100]:
# ## DO NOT NEED if there are no sub_sub_sub_collections
# ## Export file to excel with the header

# digitalhub_sub_col_sub_col_sub_col_sub_col_df.to_excel("outputs/digitalhub_sub_col_sub_col_sub_col_sub_col_df.xlsx",  header=True)

# ## Checked: No problems

In [101]:
# ## DO NOT NEED if there are no sub_sub_sub_collections
# digitalhub_sub_col_sub_col_sub_col_sub_col_df.columns

In [102]:
## If there are sub_sub_sub_collections, use: digitalhub_sub_col_sub_col_sub_col_sub_col_df
## If there are no sub_sub_sub_collections, use: digitalhub_sub_col_sub_col_sub_col_df

digitalhub_sub_col_sub_col_sub_col_df.drop(columns=['level_0', 'index'], inplace=True)

In [103]:
## Re-order Columns
## If there are sub_sub_sub_collections, use: digitalhub_sub_col_sub_col_sub_col_sub_col_df
## If there are no sub_sub_sub_collections, use: digitalhub_sub_col_sub_col_sub_col_df

re_ordered_df = digitalhub_sub_col_sub_col_sub_col_df.reindex(columns=['Level Type',
                                                                               'Level Number', 
                                                                               'Community_ID',
                                                                               'Sub_Collection_ID',
                                                                               'Sub_Sub_Collection_ID',
                                                                               'Sub_Sub_Sub_Collection_ID',
                                                                               'Sub_Sub_Sub_Sub_Collection_ID',
                                                                               'Record',
                                                                               'Member_List',
                                                                               'Member_List_Count',
                                                                               'Title',
                                                                               'uri',
                                                                               'Id',
                                                                               'Keyword',
                                                                               'Resource type(s)',
                                                                               'Rights',
                                                                               'Creator',
                                                                               'Contributor',
                                                                               'Description',
                                                                               'Abstract',
                                                                               'Original Bibliographic Citation',
                                                                               'Related ULR',
                                                                               'Publisher',
                                                                               'Date Created',
                                                                               'Original Identifier',
                                                                               'Language',
                                                                               'Subject: MESH',
                                                                               'Subject: LCSH',
                                                                               'Subject: Geographic Name',
                                                                               'Subject: Name',
                                                                               'Location',
                                                                               'Digital Origin',
                                                                               'URI',
                                                                               'Acknowledgements',
                                                                               'Grants And Funding',
                                                                               'DOI', 
                                                                               'ARK', 
                                                                               'File Size', 
                                                                               'File Format', 
                                                                               'download',
                                                                               'Multi-page?',
                                                                               'members',
                                                                               'community_rowid',
                                                                               'sub_collection_rowid',
                                                                               'sub_sub_collection_rowid',
                                                                               'sub_sub_sub_collection_rowid'
                                                                              ])


re_ordered_df.sort_values(by = ['Community_ID',
                                'community_rowid', 
                                'Sub_Collection_ID', 
                                'sub_collection_rowid',
                                'Sub_Sub_Collection_ID',
                                'sub_sub_collection_rowid',
                                'Sub_Sub_Sub_Collection_ID',
                                'sub_sub_sub_collection_rowid',
                                'Sub_Sub_Sub_Sub_Collection_ID'], ascending = [True, 
                                                                               True, 
                                                                               True, 
                                                                               True, 
                                                                               True, 
                                                                               True, 
                                                                               True, 
                                                                               True, 
                                                                               True], inplace=True)

In [104]:
re_ordered_df.head()

,Level Type,Level Number,Community_ID,Sub_Collection_ID,Sub_Sub_Collection_ID,Sub_Sub_Sub_Collection_ID,Sub_Sub_Sub_Sub_Collection_ID,Record,Member_List,Member_List_Count,...,ARK,File Size,File Format,download,Multi-page?,members,community_rowid,sub_collection_rowid,sub_sub_collection_rowid,sub_sub_sub_collection_rowid
44,Community,1,074eef38-f1fa-48bb-aa1a-dd895b73d2b1,None,None,None,NaN,NaN,[],0.0,...,None,NaN,None,None,None,[],18.0,NaN,NaN,NaN
48,Community,1,07b25bee-4a47-466a-b9b8-70d7a392fab0,07b25bee-4a47-466a-b9b8-70d7a392fab0,None,None,NaN,NaN,"[a97853c7-82d3-408c-95d8-c72a04acb41f, b0a1fa1...",2.0,...,None,NaN,None,None,False,"[{'Multi-page?': False, 'Title': ['James A. Co...",17.0,9.0,NaN,NaN
982,Sub_Collection,2,07b25bee-4a47-466a-b9b8-70d7a392fab0,a97853c7-82d3-408c-95d8-c72a04acb41f,a97853c7-82d3-408c-95d8-c72a04acb41f,None,NaN,NaN,"[aed04ba0-5b58-4a10-94de-cc062d44bcfc, b37f04a...",5.0,...,None,NaN,None,None,False,"[{'Multi-page?': False, 'Title': ['12th Genera...",17.0,51.0,NaN,NaN
1055,Sub_Collection,2,07b25bee-4a47-466a-b9b8-70d7a392fab0,b0a1fa15-eb29-416f-b452-e81b39f0f26b,b0a1fa15-eb29-416f-b452-e81b39f0f26b,None,NaN,NaN,"[bc454378-fa71-40a4-b7dc-d671bdb9a2f2, 1a3c738...",9.0,...,None,NaN,None,None,False,"[{'Multi-page?': False, 'Title': ['12th Genera...",17.0,50.0,NaN,NaN
114,Sub_Sub_Collection,3,07b25bee-4a47-466a-b9b8-70d7a392fab0,a97853c7-82d3-408c-95d8-c72a04acb41f,102d9812-55fa-4aeb-b910-04e534f7a7ef,None,NaN,NaN,"[d2dfa198-1da0-4b8f-9d28-c70f786f89bb, a5813d8...",4.0,...,None,NaN,None,None,False,"[{'Title': [' Portrait of James A. Conner'], '...",NaN,51.0,0.0,NaN


In [105]:
## Export file to excel with the header

re_ordered_df.to_excel("outputs/re_ordered_df.xlsx", index= False, header=True)

## Checked: No problems

In [116]:
## Upload a .csv DigitalHub Collections that will become Prism Communities

digitalhub_community_path = "data/2022_08-01 DigitalHub Collection Migration Plan_Communities Only.csv"

## Read the CSV file and store into Pandas DataFrame 
digitalhub_community_shape_df = pd.read_csv(digitalhub_community_path , encoding = "ISO-8859-1", na_values=['NULL', '<NA>'])

## encoding = "ISO-8859-1", na_values=['NULL', '<NA>']

#Change the column names to lower case with underscore for spaces
digitalhub_community_shape_df.columns =  digitalhub_community_shape_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")
digitalhub_community_shape_df.head()

## Checked: No problems

C:\Users\keg827\AppData\Local\Temp\ipykernel_30316\593689258.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  digitalhub_community_shape_df.columns =  digitalhub_community_shape_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")
C:\Users\keg827\AppData\Local\Temp\ipykernel_30316\593689258.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  digitalhub_community_shape_df.columns =  digitalhub_community_shape_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")


,ï»¿description,community,access:_visibility,access:_member_policy,access:_record_policy,access:_owned_by,access:_reader,id,metadata:_title,metadata:_description,metadata:_type,metadata:_website,dh_id,number_of_collections,collection_name,child-collection_name_not_present_in_prism,number_of_items,item-level_id,item-level_doi,unnamed:_19
0,Community,Biostatistics Collaboration Center Lecture Series,Public,Open,Open,"akl5916, ccb638, tlm488",NaN,biostatistics-collaboration-center-lecture-series,Biostatistics Collaboration Center Lecture Series,Free introductory lectures on biostatistics in...,project,http://www.feinberg.northwestern.edu/sites/bc...,2cc92425-b656-47ea-a3b4-825405ee6088,6,NaN,NaN,22,NaN,NaN,NaN
1,Community,Center for Community Health,Public,Open,Open,"kah2923, gmr244",NaN,center-for-community-health,Center for Community Health,NaN,organization,https://www.feinberg.northwestern.edu/sites/cch/,ae0b945c-d0d4-45bb-a0fc-263c7afca49e,0,NaN,NaN,2,NaN,NaN,NaN
2,Community,Center for Biomedical Informatics and Data Sci...,Public,Open,Open,"jbs642, aco454, nds616, ssv0467, skh8463",NaN,center-for-biomedical-informatics-and-data-sci...,Center for Biomedical Informatics and Data Sci...,The Center for Data Science and Informatics (C...,organization,https://www.feinberg.northwestern.edu/sites/au...,2828d181-5de7-4567-ba8d-d6bdb72b625f,2,NaN,NaN,23,NaN,NaN,NaN
3,Community,Dialogues in Oncofertility,Public,Open,Open,"lma467, bkm413",NaN,dialogues-in-oncofertility,Dialogues in Oncofertility,Dialogues in Oncofertility is a record of scho...,project,https://oncofertility.msu.edu/,44857499-5126-47bd-a226-5fbf291abaf6,0,NaN,NaN,7,NaN,NaN,NaN
4,Community,Galter Health Sciences Library & Learning Center,Public,Open,Open,"pls360, kal660, keg827",NaN,galter-health-sciences-library-&-learning-center,Galter Health Sciences Library & Learning Center,The Galter Health Sciences Library & Learning ...,organization,https://galter.northwestern.edu/,fj2362114,10,NaN,NaN,68,NaN,NaN,NaN


In [117]:
## Merge in fields from DigitalHub Communities spreadsheet
#re_ordered_df
#digitalhub_community_shape_df

merged_df = re_ordered_df.merge(digitalhub_community_shape_df, how = 'left', left_on='Id', right_on='dh_id')
merged_df.head()

,Level Type,Level Number,Community_ID,Sub_Collection_ID,Sub_Sub_Collection_ID,Sub_Sub_Sub_Collection_ID,Sub_Sub_Sub_Sub_Collection_ID,Record,Member_List,Member_List_Count,...,metadata:_type,metadata:_website,dh_id,number_of_collections,collection_name,child-collection_name_not_present_in_prism,number_of_items,item-level_id,item-level_doi,unnamed:_19
0,Community,1,074eef38-f1fa-48bb-aa1a-dd895b73d2b1,None,None,None,NaN,NaN,[],0.0,...,organization,https://www.feinberg.northwestern.edu/sites/ca...,074eef38-f1fa-48bb-aa1a-dd895b73d2b1,0.0,NaN,NaN,0.0,NaN,NaN,NaN
1,Community,1,07b25bee-4a47-466a-b9b8-70d7a392fab0,07b25bee-4a47-466a-b9b8-70d7a392fab0,None,None,NaN,NaN,"[a97853c7-82d3-408c-95d8-c72a04acb41f, b0a1fa1...",2.0,...,topic,https://sites.northwestern.edu/twelfthgeneralh...,07b25bee-4a47-466a-b9b8-70d7a392fab0,2.0,NaN,NaN,753.0,NaN,NaN,NaN
2,Sub_Collection,2,07b25bee-4a47-466a-b9b8-70d7a392fab0,a97853c7-82d3-408c-95d8-c72a04acb41f,a97853c7-82d3-408c-95d8-c72a04acb41f,None,NaN,NaN,"[aed04ba0-5b58-4a10-94de-cc062d44bcfc, b37f04a...",5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sub_Collection,2,07b25bee-4a47-466a-b9b8-70d7a392fab0,b0a1fa15-eb29-416f-b452-e81b39f0f26b,b0a1fa15-eb29-416f-b452-e81b39f0f26b,None,NaN,NaN,"[bc454378-fa71-40a4-b7dc-d671bdb9a2f2, 1a3c738...",9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sub_Sub_Collection,3,07b25bee-4a47-466a-b9b8-70d7a392fab0,a97853c7-82d3-408c-95d8-c72a04acb41f,102d9812-55fa-4aeb-b910-04e534f7a7ef,None,NaN,NaN,"[d2dfa198-1da0-4b8f-9d28-c70f786f89bb, a5813d8...",4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
merged_df.columns

Index(['Level Type', 'Level Number', 'Community_ID', 'Sub_Collection_ID',
       'Sub_Sub_Collection_ID', 'Sub_Sub_Sub_Collection_ID',
       'Sub_Sub_Sub_Sub_Collection_ID', 'Record', 'Member_List',
       'Member_List_Count', 'Title', 'uri', 'Id', 'Keyword',
       'Resource type(s)', 'Rights', 'Creator', 'Contributor', 'Description',
       'Abstract', 'Original Bibliographic Citation', 'Related ULR',
       'Publisher', 'Date Created', 'Original Identifier', 'Language',
       'Subject: MESH', 'Subject: LCSH', 'Subject: Geographic Name',
       'Subject: Name', 'Location', 'Digital Origin', 'URI',
       'Acknowledgements', 'Grants And Funding', 'DOI', 'ARK', 'File Size',
       'File Format', 'download', 'Multi-page?', 'members', 'community_rowid',
       'sub_collection_rowid', 'sub_sub_collection_rowid',
       'sub_sub_sub_collection_rowid', 'ï»¿description', 'community',
       'access:_visibility', 'access:_member_policy', 'access:_record_policy',
       'access:_owned_by', '

In [119]:
## Drop extra columns

drop_columns = ['ï»¿description','community','metadata:_title', 'metadata:_description', 'dh_id',
           'number_of_collections', 'collection_name',       'child-collection_name_not_present_in_prism', 
           'number_of_items','item-level_id', 'item-level_doi', 'unnamed:_19']

merged_df.drop(columns= drop_columns, inplace=True)
merged_df.head()

,Level Type,Level Number,Community_ID,Sub_Collection_ID,Sub_Sub_Collection_ID,Sub_Sub_Sub_Collection_ID,Sub_Sub_Sub_Sub_Collection_ID,Record,Member_List,Member_List_Count,...,sub_sub_collection_rowid,sub_sub_sub_collection_rowid,access:_visibility,access:_member_policy,access:_record_policy,access:_owned_by,access:_reader,id,metadata:_type,metadata:_website
0,Community,1,074eef38-f1fa-48bb-aa1a-dd895b73d2b1,None,None,None,NaN,NaN,[],0.0,...,NaN,NaN,Public,Open,Open,jnb235; gas162; ers050; lah315,NaN,northwestern-pepper-center,organization,https://www.feinberg.northwestern.edu/sites/ca...
1,Community,1,07b25bee-4a47-466a-b9b8-70d7a392fab0,07b25bee-4a47-466a-b9b8-70d7a392fab0,None,None,NaN,NaN,"[a97853c7-82d3-408c-95d8-c72a04acb41f, b0a1fa1...",2.0,...,NaN,NaN,Public,Open,Open,kal660,NaN,12th-general-hospital-collection,topic,https://sites.northwestern.edu/twelfthgeneralh...
2,Sub_Collection,2,07b25bee-4a47-466a-b9b8-70d7a392fab0,a97853c7-82d3-408c-95d8-c72a04acb41f,a97853c7-82d3-408c-95d8-c72a04acb41f,None,NaN,NaN,"[aed04ba0-5b58-4a10-94de-cc062d44bcfc, b37f04a...",5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sub_Collection,2,07b25bee-4a47-466a-b9b8-70d7a392fab0,b0a1fa15-eb29-416f-b452-e81b39f0f26b,b0a1fa15-eb29-416f-b452-e81b39f0f26b,None,NaN,NaN,"[bc454378-fa71-40a4-b7dc-d671bdb9a2f2, 1a3c738...",9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sub_Sub_Collection,3,07b25bee-4a47-466a-b9b8-70d7a392fab0,a97853c7-82d3-408c-95d8-c72a04acb41f,102d9812-55fa-4aeb-b910-04e534f7a7ef,None,NaN,NaN,"[d2dfa198-1da0-4b8f-9d28-c70f786f89bb, a5813d8...",4.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
merged_df.columns

Index(['Level Type', 'Level Number', 'Community_ID', 'Sub_Collection_ID',
       'Sub_Sub_Collection_ID', 'Sub_Sub_Sub_Collection_ID',
       'Sub_Sub_Sub_Sub_Collection_ID', 'Record', 'Member_List',
       'Member_List_Count', 'Title', 'uri', 'Id', 'Keyword',
       'Resource type(s)', 'Rights', 'Creator', 'Contributor', 'Description',
       'Abstract', 'Original Bibliographic Citation', 'Related ULR',
       'Publisher', 'Date Created', 'Original Identifier', 'Language',
       'Subject: MESH', 'Subject: LCSH', 'Subject: Geographic Name',
       'Subject: Name', 'Location', 'Digital Origin', 'URI',
       'Acknowledgements', 'Grants And Funding', 'DOI', 'ARK', 'File Size',
       'File Format', 'download', 'Multi-page?', 'members', 'community_rowid',
       'sub_collection_rowid', 'sub_sub_collection_rowid',
       'sub_sub_sub_collection_rowid', 'access:_visibility',
       'access:_member_policy', 'access:_record_policy', 'access:_owned_by',
       'access:_reader', 'id', 'metadat

In [123]:
## Final Re-order of columns

## Re-order Columns
## If there are sub_sub_sub_collections, use: digitalhub_sub_col_sub_col_sub_col_sub_col_df
## If there are no sub_sub_sub_collections, use: digitalhub_sub_col_sub_col_sub_col_df

final_re_ordered_df = merged_df.reindex(columns=['Level Type',
                                                   'Level Number', 
                                                   'Community_ID',
                                                   'Sub_Collection_ID',
                                                   'Sub_Sub_Collection_ID',
                                                   'Sub_Sub_Sub_Collection_ID',
                                                   'Sub_Sub_Sub_Sub_Collection_ID',
                                                   'Record',
                                                   'Member_List',
                                                   'Member_List_Count',
                                                   'Title',
                                                   'uri',
                                                   'Id',
                                                   'access:_visibility',
                                                   'access:_member_policy', 
                                                   'access:_record_policy', 
                                                   'access:_owned_by', 
                                                   'access:_reader',
                                                   'id', 
                                                   'metadata:_type', 
                                                   'metadata:_website'
                                                   'Keyword',
                                                   'Resource type(s)',
                                                   'Rights',
                                                   'Creator',
                                                   'Contributor',
                                                   'Description',
                                                   'Abstract',
                                                   'Original Bibliographic Citation',
                                                   'Related ULR',
                                                   'Publisher',
                                                   'Date Created',
                                                   'Original Identifier',
                                                   'Language',
                                                   'Subject: MESH',
                                                   'Subject: LCSH',
                                                   'Subject: Geographic Name',
                                                   'Subject: Name',
                                                   'Location',
                                                   'Digital Origin',
                                                   'URI',
                                                   'Acknowledgements',
                                                   'Grants And Funding',
                                                   'DOI', 
                                                   'ARK', 
                                                   'File Size', 
                                                   'File Format', 
                                                   'download',
                                                   'Multi-page?',
                                                   'members',
                                                   'community_rowid',
                                                   'sub_collection_rowid',
                                                   'sub_sub_collection_rowid',
                                                   'sub_sub_sub_collection_rowid'
                                                  ])

final_re_ordered_df.rename(columns={'access:_visibility': 'Prism Visibility',
                                       'access:_member_policy': 'Prism Member Policy', 
                                       'access:_record_policy': 'Prism Record Policy', 
                                       'access:_owned_by': 'Prism Owners', 
                                       'access:_reader': 'Prism Readers',
                                       'id': 'Prism_Link', 
                                       'metadata:_type': 'Prism Community Type', 
                                       'metadata:_website': 'Prism Community Website'
                                    })

final_re_ordered_df.sort_values(by = ['Community_ID',
                                'community_rowid', 
                                'Sub_Collection_ID', 
                                'sub_collection_rowid',
                                'Sub_Sub_Collection_ID',
                                'sub_sub_collection_rowid',
                                'Sub_Sub_Sub_Collection_ID',
                                'sub_sub_sub_collection_rowid',
                                'Sub_Sub_Sub_Sub_Collection_ID'], ascending = [True, 
                                                                               True, 
                                                                               True, 
                                                                               True, 
                                                                               True, 
                                                                               True, 
                                                                               True, 
                                                                               True, 
                                                                               True], inplace=True)
final_re_ordered_df.head()

,Level Type,Level Number,Community_ID,Sub_Collection_ID,Sub_Sub_Collection_ID,Sub_Sub_Sub_Collection_ID,Sub_Sub_Sub_Sub_Collection_ID,Record,Member_List,Member_List_Count,...,ARK,File Size,File Format,download,Multi-page?,members,community_rowid,sub_collection_rowid,sub_sub_collection_rowid,sub_sub_sub_collection_rowid
0,Community,1,074eef38-f1fa-48bb-aa1a-dd895b73d2b1,None,None,None,NaN,NaN,[],0.0,...,None,NaN,None,None,None,[],18.0,NaN,NaN,NaN
1,Community,1,07b25bee-4a47-466a-b9b8-70d7a392fab0,07b25bee-4a47-466a-b9b8-70d7a392fab0,None,None,NaN,NaN,"[a97853c7-82d3-408c-95d8-c72a04acb41f, b0a1fa1...",2.0,...,None,NaN,None,None,False,"[{'Multi-page?': False, 'Title': ['James A. Co...",17.0,9.0,NaN,NaN
2,Sub_Collection,2,07b25bee-4a47-466a-b9b8-70d7a392fab0,a97853c7-82d3-408c-95d8-c72a04acb41f,a97853c7-82d3-408c-95d8-c72a04acb41f,None,NaN,NaN,"[aed04ba0-5b58-4a10-94de-cc062d44bcfc, b37f04a...",5.0,...,None,NaN,None,None,False,"[{'Multi-page?': False, 'Title': ['12th Genera...",17.0,51.0,NaN,NaN
3,Sub_Collection,2,07b25bee-4a47-466a-b9b8-70d7a392fab0,b0a1fa15-eb29-416f-b452-e81b39f0f26b,b0a1fa15-eb29-416f-b452-e81b39f0f26b,None,NaN,NaN,"[bc454378-fa71-40a4-b7dc-d671bdb9a2f2, 1a3c738...",9.0,...,None,NaN,None,None,False,"[{'Multi-page?': False, 'Title': ['12th Genera...",17.0,50.0,NaN,NaN
4,Sub_Sub_Collection,3,07b25bee-4a47-466a-b9b8-70d7a392fab0,a97853c7-82d3-408c-95d8-c72a04acb41f,102d9812-55fa-4aeb-b910-04e534f7a7ef,None,NaN,NaN,"[d2dfa198-1da0-4b8f-9d28-c70f786f89bb, a5813d8...",4.0,...,None,NaN,None,None,False,"[{'Title': [' Portrait of James A. Conner'], '...",NaN,51.0,0.0,NaN


In [124]:
## Export file to excel with the header

final_re_ordered_df.to_excel("outputs/final_re_ordered_df.xlsx", index= False, header=True)

## Checked: No problems

### Communitiy Requests in Prism

##### not yet complete

In [ ]:
## API request from Prism for public communities

url = 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities'

params={
        'accept': 'application/json',
        'access_token': api_key_prism
       }

community_response = requests.get(url, params=params)
print(community_response.url)
print(community_response.status_code)

## Use the .json() to return a JSON object of the result (if the result was written in JSON format)
## The result is a python dictionary
## Resource: https://www.geeksforgeeks.org/response-methods-python-requests/
communtiy_response_json = community_response.json()

## Next, pass the Python dictionary to json.dumps() function, which will return a string. 
print(json.dumps(communtiy_response_json, indent=4, sort_keys=True))

In [ ]:
## Using the python dictionary, go two levels deep (i.e. Hits, then  hits) to grab the community record data
community_records = communtiy_response_json['hits']['hits']
print(community_records)

In [ ]:
## Use pandas json_normalize() to normalize semi-structured JSON data into a flat table.
## Max number of levels(depth of dict) to normalize. if None, normalizes all levels.
## Resource: https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html
prism_community_nested_df = pd.json_normalize(community_records)
prism_community_nested_df.head()

In [ ]:
## Export file to excel, without the Pandas index, but with the header

prism_community_nested_df.to_excel("outputs/public_communities.xlsx", index=False, header=True)

In [ ]:
## API request from Prism for restricted communities

url = 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities'

params={
        'accept': 'application/json',
        "access": {
            "visibility": 'restricted'
            },
        'access_token': api_key_prism
       }

restricted_community_response = requests.get(url, params=params)
print(restricted_community_response.url)
print(restricted_community_response.status_code)

## Use the .json() to return a JSON object of the result (if the result was written in JSON format)
## The result is a python dictionary
## Resource: https://www.geeksforgeeks.org/response-methods-python-requests/
restricted_communtiy_response_json = restricted_community_response.json()

## Next, pass the Python dictionary to json.dumps() function, which will return a string. 
print(json.dumps(restricted_communtiy_response_json, indent=4, sort_keys=True))

In [ ]:
## Using the python dictionary, go two levels deep (i.e. Hits, then  hits) to grab the community record data
restricted_community_records = restricted_communtiy_response_json['hits']['hits']
print(restricted_community_records)

In [ ]:
## Use pandas json_normalize() to normalize semi-structured JSON data into a flat table.
## Max number of levels(depth of dict) to normalize. if None, normalizes all levels.
## Resource: https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html
prism_restricted_community_nested_df = pd.json_normalize(restricted_community_records)
prism_restricted_community_nested_df.head()